%%javascript
IPython.notebook.kernel.execute("clear_output()")

In [ ]:
# Importación de módulos del sistema
import os
import glob
import copy
import time
import datetime
import math

# Importación de librerías para manipulación de datos y cálculos numéricos
import numpy as np
import pandas as pd

# Importación de la librería Obspy para procesamiento de datos sísmicos
import obspy
from obspy import read, Trace, UTCDateTime as UTC
from obspy.signal.util import _npts2nfft
from obspy.signal.invsim import cosine_taper
from obspy.signal import cross_correlation as occ
from obspy.signal.cross_correlation import correlation_detector
from obspy.signal.filter import bandpass, lowpass
from obspy.signal.regression import linear_regression
from obspy.core.util.base import _get_function_from_entry_point
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.signal.trigger import (classic_sta_lta, classic_sta_lta_py, 
                                  plot_trigger, z_detect, recursive_sta_lta, 
                                  carl_sta_trig)

# Importación de la librería Scipy para cálculos científicos y procesamiento de señales
import scipy
from scipy import signal
from scipy import stats as st
import scipy.stats as stats
from scipy.signal import (hilbert, correlate, correlation_lags, argrelextrema, 
                          savgol_filter)
from scipy.stats import (percentileofscore, gaussian_kde, kurtosis, skew, 
                         pearsonr, norm)
from scipy.fftpack import (fft, fftfreq, fftshift, ifft, next_fast_len)

# Importación de módulos para manejo de fechas y zonas horarias
from datetime import tzinfo, timezone

# Importación de la librería Colorama para formato de texto en la terminal
from colorama import init, Fore, Style

# Importación de la librería Matplotlib para visualización de datos
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.colors import ListedColormap

# Importación de la librería Seaborn para visualización de datos con Matplotlib
import seaborn as sns
sns.set()  # Configuración del estilo por defecto de Seaborn

# Importación de la librería Scikit-learn para algoritmos de Machine Learning
from sklearn.cluster import KMeans

# Importación de herramientas de IPython para visualización en notebooks
from IPython.display import Image

# Aquí iría el resto de tu código


In [ ]:
def normal_distribution(x, media, desv_est):
    return np.exp(-1*((x-media)**2)/(2*(desv_est**2)))/(math.sqrt(2*np.pi) * desv_est)
def contador_explosiones (signal_tr, stalta_array, trg_on, trg_off):
    detections = []
    tr_times = []
    tr_time_ini = []
    tr_time_fin = []
    explo_count = 0
    ybp_array = []
    cft_sv_array =[]
    explode = []
    for v in signal_tr.times("utcdatetime")  :
        tr_times.append([v])

        
    for v in signal_tr.data:
        ybp_array.append([v])
    
    for v in stalta_array:
        cft_sv_array.append([v])
        
    evtgr = 0  

    for i  in np.arange(len(ybp_array)):
        if cft_sv_array[i] > trg_on and  evtgr ==0:
            evtgr = 1
            tr_time_ini.append(tr_times[i])
        if cft_sv_array[i] < trg_off and evtgr ==1:
            evtgr = 0
            tr_time_fin.append(tr_times[i])
            explo_count +=1
            explode.append([explo_count-1])
 
    if len(tr_time_ini) > len(tr_time_fin):   # Verifica si tr_time_ini es más grande que tr_time_fin
        tr_time_ini = tr_time_ini[:len(tr_time_fin)]   # Reduce el tamaño de tr_time_ini al tamaño de tr_time_fin
    else:
        tr_time_fin = tr_time_fin[:len(tr_time_ini)]   # Reduce el tamaño de tr_time_fin al tamaño de tr_time_ini


    the_times = np.concatenate((explode,tr_time_ini,tr_time_fin),axis=1)
    the_times_df = pd.DataFrame(the_times, columns = ['explode','time_start','time_end'] )

    return explo_count, tr_time_ini, tr_time_fin, the_times_df

def contador_explosiones_simple (signal_tr, stalta_array, trg_on, trg_off):
    detections = []
    explo_count = 0
    ybp_array = []
    cft_sv_array =[]
    explode = []
        
    for v in signal_tr.data:
        ybp_array.append([v])
    
    for v in stalta_array:
        cft_sv_array.append([v])

    evtgr = 0  

    for i  in np.arange(len(ybp_array)):
        if (cft_sv_array[i]) > trg_on and  evtgr ==0:
            evtgr = 1
        if (cft_sv_array[i]) < trg_off and evtgr ==1:
            evtgr = 0
            explo_count +=1
            

    return explo_count

def string_to_utcdatetime(string_v):
    hay_milis = True
    string_v = str(string_v)
    y_ini =13
    y_fin = (string_v.find(',',13))
    m_fin = string_v.find(',',y_fin+1)
    d_fin = string_v.find(',',m_fin+1)
    h_fin = string_v.find(',',d_fin+1)
    mi_fin = string_v.find(',',h_fin+1)
    s_fin = string_v.find(',',mi_fin+1)
    if s_fin == -1:
        s_fin = tini.find(')',mi_fin+1)
        hay_milis = False
    else:
        mili_fin = string_v.find(')', s_fin+1)
    
    y=(int(string_v[y_ini:y_fin]))
    m=(int(string_v[y_fin+1:m_fin]))
    d=(int(string_v[m_fin+1:d_fin]))
    h=(int(string_v[d_fin+1:h_fin]))
    mi=(int(string_v[h_fin+1:mi_fin]))
    s=(int(string_v[mi_fin+1:s_fin]))
    if hay_milis:
        mili=(int(string_v[s_fin+1:mili_fin]))
        return UTCDateTime(y,m,d,h,mi,s,mili)
    else:
        return UTCDateTime(y,m,d,h,mi,s)

def similarity_component_thres(ccs, thres, num_components):
    """Return Trace with mean of ccs
    and set values to zero if number of components above threshold is not reached"""
    ccmatrix = np.array([tr.data for tr in ccs])
    header = dict(sampling_rate=ccs[0].stats.sampling_rate,
                  starttime=ccs[0].stats.starttime)
    comp_thres = np.sum(ccmatrix > thres, axis=0) >= num_components
    data = np.mean(ccmatrix, axis=0) * comp_thres
    return Trace(data=data, header=header)

def simf(ccs):
    return similarity_component_thres(ccs, 0.6, 2)

In [ ]:
def suavizado_por_convolucion(Data):
    t = np.arange(len(Data))

    #llenamos el vector de ceros
    g = np.zeros(len(Data))
    g[t<=500]= 1/1001
    g[-500:] = 1/1001
    print (g)

    # Hacemos las transformadas de fourier
    signal_tf = np.fft.fft(Data)
    g_tf = np.fft.fft(g)

    #Hacemos la convolución
    y_tf = signal_tf * g_tf
    cft_sv = np.fft.ifft(y_tf)

    return np.real(cft_sv)
def suavizado_señal(signal, ventana):
    signal = pd.Series(signal)

    # Definir la ventana móvil
    window_size = ventana
    window = signal.rolling(window_size, center=True)

    # Calcular la mediana móvil
    smooth_signal = window.median()
    #print(len(signal), len(smooth_signal))
    smooth_signal = np.nan_to_num(smooth_signal, nan=0.0)
    #print(" el valor de smooth signal en la casilla  ",1,smooth_signal[1])


    # Calcular la desviación estándar móvil
    std_signal = window.std()

    # Identificar los outliers en la señal suavizada
    outliers = smooth_signal[(smooth_signal < signal - 3*std_signal) | (smooth_signal > signal + 3*std_signal)]

    # Encontrar los máximos y mínimos relativos en la señal suavizada
    maximos = argrelextrema(smooth_signal, np.greater)[0]
    minimos = argrelextrema(smooth_signal, np.less)[0]
    extremos_relativos = np.sort(np.concatenate((maximos, minimos)))
    extremos_relativos_vector = np.zeros(len(signal))

    # Asignar los valores de smooth_signal[extremos_relativos] al vector de ceros
    extremos_relativos_vector[extremos_relativos] = smooth_signal[extremos_relativos]
    return smooth_signal, extremos_relativos, extremos_relativos_vector, maximos,minimos

# Derivada numérica (extremos relativos)
def  condicion_primer_orden( smooth_signal):
    extrems = np.zeros(len(smooth_signal))
    t = np.arange(len(smooth_signal))
    smooth_signal = np.nan_to_num(smooth_signal, nan=0.0)

    for x in t[1:-1]:
        c = x
        a = int(c - 1)
        b = int(c + 1)
        dleft = int(smooth_signal[a]) - int(smooth_signal[x])
        dright = smooth_signal[b] - smooth_signal[x]
        if dleft < 0 and dright < 0:
            extrems[c] = smooth_signal[c]
        elif dleft > 0 and dright > 0:
            extrems[c] = smooth_signal[c]
        else:
            extrems[x] = 0
    return extrems
    
def encontrar_extremos_relativos(smooth_signal):
    extrems = np.zeros(len(smooth_signal))
    t = np.arange(len(smooth_signal))
    smooth_signal = np.nan_to_num(smooth_signal, nan=0.0)

    # Calcula la derivada numérica 
    dx_Y = np.diff(smooth_signal, prepend=0, append=0)

    for x in t[1:-1]:
        # Ajuste en la condición de extremo: Cambio de signo en la derivada
        if np.sign(dx_Y[x]) != np.sign(dx_Y[x + 1]):
            extrems[x] = smooth_signal[x]

    return extrems

In [ ]:
def calculo_clusters_signal (x,y):
    n_clusters = n_Clusters
    kmeans = KMeans(n_clusters)
    x_multi = np.concatenate((x,y),axis=1)
    kmeans.fit(x_multi)
    clusters = kmeans.fit_predict(x_multi)
    cluster_array = []
    for u in clusters.data:
        cluster_array.append([u])

    return cluster_array

def calculo_clusters_signal_seismic_2 (x,x2,y, n__Clusters):
    n_clusters = n__Clusters
    kmeans = KMeans(n_clusters)
    x_multi = np.concatenate((x,x2,y),axis=1)
    kmeans.fit(x_multi)
    clusters = kmeans.fit_predict(x_multi)
    cluster_array = []
    for u in clusters.data:
        cluster_array.append([u])

    return cluster_array

def calculo_clusters_signal_seismic (x,y, n__Clusters):
    n_clusters = n__Clusters
    kmeans = KMeans(n_clusters)
    x_multi = np.concatenate((x,y),axis=1)
    kmeans.fit(x_multi)
    clusters = kmeans.fit_predict(x_multi)
    cluster_array = []
    for u in clusters.data:
        cluster_array.append([u])

    return cluster_array
    
##============================================
## Cálculo de estadísticas de conglomerados 
##============================================
def  estadisticas_conglomerados( r_multi_df):
    y_clus      = []
    y_mean      = []
    y_std       = []
    y_min       = []
    y_max       = []
    y_median    = []
    y_count     = []
    y_rcount     = []
    y_scount     = []
    array_size  = len(r_multi_df)
    minut_size = array_size/df/60
    
    for i in np.arange(n_Clusters): 
        dfd= r_multi_df[r_multi_df.c == i]
        
        y_clus.append([i])
        y_mean.append([dfd.y.mean()])
        y_std.append([dfd.y.std()])
        y_min.append([dfd.y.min()])
        y_max.append([dfd.y.max()])
        y_median.append([dfd.y.median()])
        y_count.append([dfd.y.count()])
        y_rcount.append([dfd.y.count()/array_size])
        y_scount.append([dfd.y.count()/array_size*minut_size])

    r_multi = np.concatenate((y_clus, y_mean,y_std,y_min, y_max, y_median, y_count, y_rcount, y_scount),axis=1)
    multi_prm = pd.DataFrame(r_multi, columns = ['c','ymean','ystd','ymin','ymax','ymedian','ycount', 'yrcount', 'yscount'] )

    prm_sorted = multi_prm.sort_values(by=['ymean'])
    prm_sorted = prm_sorted.reset_index(drop=True)
    return(prm_sorted)

def cluster_N (mul_df, Cluster):
    c_c = prm_sorted_signal.iloc[C_idx:,0]
    t_c    = []
    x_c    = HDF_zeros
    y_c    = HDF_zeros
    y_sv_c = HDF_zeros
    index_c =[]
    for j in np.arange(len(HDF_zeros)):
        t_c.append([0])
    dfd_i   = mul_df[mul_df.c == c_c].index
    dfd_d = mul_df[mul_df.c == c_c]

    return dfd_d, dfd_i

def cluster_N_extrem (mul_df, prm_sorted,C_idx,plus):
    c_c = prm_sorted_signal.iloc[C_idx:,0]
    t_c    = []
    x_c    = EHZ_zeros
    y_c    = EHZ_zeros
    y_sv_c = EHZ_zeros
    
    index_c =[]
    cond = True 
    if plus== "=":
        c_c = prm_sorted_signal.iloc[C_idx,0]
    elif plus == "+":
        c_c = prm_sorted_signal.iloc[C_idx:,0]
    elif plus == "-":
        c_c = prm_sorted_signal.iloc[:C_idx,0]
    if plus == "+" or plus =="-":
        dfd_i   = mul_df[mul_df['c'].isin(c_c)].index
        dfd_d = mul_df[mul_df['c'].isin(c_c)]
    else:
        dfd_i   = mul_df[mul_df.c == c_c].index
        dfd_d = mul_df[mul_df.c == c_c]
        
    return dfd_d, dfd_i

def cluster_N_extrems (mul_df, prm_sorted,c_idx):
    C_idx = int(c_idx)
    seq = []
    for x in np.arange(n_Clusters):
        seq.append(x)
    result = []
    ran_in = n_Clusters-C_idx

    for i in range(C_idx):
        result.append(seq[i])
    for i in range(ran_in,n_Clusters):
        result.append(seq[i])

    c_c = []
    for i in prm_sorted_signal.iloc[result,0].values:
        c_c.append(int(i))
    #for i in prm_sorted_signal.iloc[up_l:,0].values:
    #    c_c.append(int(i))
        
    #for i in prm_sorted_signal.iloc[:lo_l,0].values:
    #    c_c.append(int(i))
    
    dfd_i   = mul_df[mul_df['c'].isin(c_c)].index
    dfd_d = mul_df[mul_df['c'].isin(c_c)]
    
    return dfd_d, dfd_i
    
def cluster_N_extrems_seismic (mul_df, prm_sorted,c_idx):
    C_idx = int(c_idx)
    seq = []
    for x in np.arange(n_Clusters+n_plus_Clusters):
        seq.append(x)
    result = []
    ran_in = n_Clusters+n_plus_Clusters-C_idx

    for i in range(C_idx):
        result.append(seq[i])
    for i in range(ran_in,n_Clusters):
        result.append(seq[i])

    c_c = []
    for i in prm_sorted_signal.iloc[result,0].values:
        c_c.append(int(i))
    #for i in prm_sorted_signal.iloc[up_l:,0].values:
    #    c_c.append(int(i))
        
    #for i in prm_sorted_signal.iloc[:lo_l,0].values:
    #    c_c.append(int(i))
    
    dfd_i   = mul_df[mul_df['c'].isin(c_c)].index
    dfd_d = mul_df[mul_df['c'].isin(c_c)]
    return dfd_d, dfd_i

In [ ]:
pick_up = 600
pick_up_to = 150
fmin_HDF=0.5
fmax_HDF=20
fmin_EHZ = 5#0.5 #5 #.5
fmax_EHZ = 15#5   #15 #7
fmin_seismic = 0.2
fmax_seismic = 7
df = 50
tiempo_entre_explosiones_min= 3
x_Clusters = 5
n_Clusters = 25 #29#25#19
n_plus_Clusters = 8#0 #8
pascal_multiplicador = 0.0002777
dif_ini_delay = 30
t_m = df
cero_period = tiempo_entre_explosiones_min
cross_treshold = .60

# Procesamiento de señal sísmica


In [ ]:
pick = UTC('2022-09-17T0:00:00')


st__HDF = read("GI.FG12.02.BDF.D.2022.260.mseed")
st__EHZ = read("GI.FG12.00.BHZ.D.2022.260.mseed")

tr_HDF = st__HDF[0]
tr_EHZ = st__EHZ[0]

tr_tr = range(0,tr_EHZ.stats.npts,pick_up)
l_t_i = tr_EHZ.stats.starttime


times = np.arange(tr_EHZ.times("utcdatetime")[0], tr_EHZ.times("utcdatetime")[-1], pick_up)
ii = 1
l_t_i = times[0]
detecciones = []
det_inicio = []
det_final = []
det_tr_inicio =[]
det_tr_fin = []

seismic_inicio = []
seismic_fin    = []
for t in times[1:]:
    print(" ") 
    print(" ") 
    print(" ") 
    print("==========================")
    print(ii,t)
    print(" ") 
    print(" ") 
    ii +=1
    t_i = t
    #print(t_i)
    #print(l_t_i, t_i)
    st_HDF_trimmed = st__HDF.copy()
    st_EHZ_trimmed = st__EHZ.copy()
    st_EHZ = st_EHZ_trimmed.trim(l_t_i, t_i)
    st_HDF = st_HDF_trimmed.trim(l_t_i, t_i)
    st_HDF = st_HDF.filter('bandpass', freqmin= fmin_HDF, freqmax=fmax_HDF, corners=4, zerophase=True)
    st_EHZ = st_EHZ.filter('bandpass', freqmin= fmin_seismic, freqmax=fmax_seismic, corners=2, zerophase=True)

    tr_HDF = st_HDF[0]
    tr_EHZ = st_EHZ[0]
    l_t_i =  t_i

    serie  = tr_EHZ.data 
    signal = pd.Series(serie)
    
    ss, er, erv,s_mx,s_mn = suavizado_señal(signal,7)
    ss2, er2, erv2,s_mx2,s_mn2 = suavizado_señal(signal,25)
    

    dist = np.diff(er)

    #===========================================
    # Derivada numérica (extremos relativos)
    extrems = encontrar_extremos_relativos(ss)
    extrems2 = encontrar_extremos_relativos(ss2)
    
    t = np.arange(len(extrems))
    print("t", t)
    print("extrems", extrems)
    print("len(t)", len(t), "extrems", len(extrems))
    plt.figure(figsize=(35,15))
    plt.plot(signal, color='#1E90FF', alpha=.5, label='Señal original', zorder=0)
    plt.plot(ss, color='#FF4500', alpha=.5, label= "onda suavizada", zorder=1)
    plt.legend(fontsize=25)

    # Configurar las etiquetas de los ejes
    plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
    plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)
    
    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.show()

    plt.figure(figsize=(35,15))
    plt.plot(signal, color='#1E90FF', alpha=.5, label='Señal original', zorder=0)
    plt.plot(ss2, color='#FF4500', alpha=.5, label= "onda suavizada", zorder=1)
    plt.legend(fontsize=25)
    
    # Configurar las etiquetas de los ejes
    plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
    plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)
    
    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.show()

    ##########################################
    ##########################################
    ##                                      ##
    ##           Clústers                   ##
    ##                                      ##
    ##########################################
    ##########################################

    t = np.arange(len(tr_EHZ.data))
    y_array      = []
    y_sv_array   = []
    x_array      = []
    x2_array     = []
    t_array      = []
    for r in extrems.data:
        x_array.append([r])

    for r2 in extrems2.data:
        x2_array.append([r2])
        
    for s in tr_EHZ.data:
        y_array.append([s])
    for s in ss:
        y_sv_array.append([s])

    for u in t:
        t_array.append([u])

    n_clusters = n_Clusters + n_plus_Clusters
    kmeans = KMeans(n_clusters)
    
    cluster_array = calculo_clusters_signal_seismic_2(x_array,x2_array,y_array,n_clusters)
    x_multi           = np.concatenate((t_array,x_array,y_array,y_sv_array, cluster_array),axis=1)
    multi_df          = pd.DataFrame(x_multi, columns = ['t','x','y','y_sv','c'] )
    prm_sorted_signal = estadisticas_conglomerados(multi_df)
    
##===================================================
## Optimización de detecciones descontando outliers
##===================================================
    k = int((n_clusters - 1) / 2)

    # Generar los valores de la secuencia en un rango de 1 a n
    sequence = list(range(1, k + 1))

    # Ordenar la secuencia de mayor a menor
    sequence.sort(reverse=True)

    t_distancia_avg = []
    t_conteo        = []
    t_cluster       = []


    # Iterar en un ciclo for
    for x_clusters in sequence:
        t_inicio        = []
        t_inicio_val    = []
        t_final         = []
        t_final_val     = []
        t_distancia     = []
        
        c_df, c_idx = cluster_N_extrems_seismic (multi_df, prm_sorted_signal,x_clusters)
        t = c_df.t.values
        y = c_df.y.values
    
        last_t = t[0]
        very_last = t[0]
        for ji in t[1:]:
            if (ji - very_last) >= t_m*cero_period and (very_last - last_t) <= t_m:
                last_t = ji
                very_last_t = ji
            elif (ji - very_last) >= t_m*cero_period and (very_last - last_t) > t_m:
                t_inicio.append([last_t])
                t_inicio_val.append(y_array[int(last_t)])
                t_final.append([very_last])
                t_final_val.append(y_array[int(very_last)])
                t_distancia.append(abs(last_t-very_last))

                last_t = ji
                if ji == t[-1]:
                    t_inicio.append([last_t])
                    t_inicio_val.append([y_array[int(last_t)]])
                    t_final.append([ji])
                    t_final_val.append(y_array[int(ji)])
                    t_distancia.append(abs(last_t-ji))
                    break
            if ji == t[-1]:
                t_inicio.append([last_t])
                t_inicio_val.append([y_array[int(last_t)]])
                t_final.append([ji])
                t_final_val.append(y_array[int(ji)])
                t_distancia.append(abs(last_t-ji))
                break
            very_last = ji
            
        t_conteo.append(len(t_final))
        t_cluster.append(x_clusters)
        t_distancia_avg.append(np.mean(t_distancia))
        print(x_clusters, len(t_final),np.mean(t_distancia)/t_m )
        
    t_conteos_max = np.argmax(t_conteo)
    
    for i in range(len(t_conteo)):
        if all(x == t_conteo[0] for x in t_conteo):
            print(" aplico 1")
            max_index = round(len(t_conteo)/2, 0)
            break

        if (t_cluster[i] == t_cluster[t_conteos_max] 
            and   t_conteo[t_conteos_max] <=20 
            and (t_conteo[t_conteos_max]-t_conteo[t_conteos_max+1])>5
            and t_conteo[i-1]!=1
            and t_conteo[i]>1 
            and t_conteo[i-1]==1 
            and i <=1
           ):
            print(" aplico 2")
            max_index = t_cluster[i]
            break
                    
        if i == len(t_conteo) - 1:
            print(" aplico 3")
            max_index = t_cluster[i-1]
            break
                
        if (i < len(t_conteo)-3 and i >=2
            and t_conteo[i-2] ==1
            and t_conteo[i-1] >1 
           ):
            max_index = t_cluster[i]
            if t_conteo[max_index] >20:
                print(" aplico 4")
                max_index = t_cluster[i+1]
            elif t_distancia_avg[max_index] > 45 and t_distancia_avg[max_index] > t_distancia_avg[max_index+1]:
                print(" aplico 5")
                if i >1 and t_conteo[i-1] ==1:
                    print(" aplico 5.1")
                    max_index=t_cluster[i+2]
                elif t_conteo[i+1] >1 and t_conteo[i+2]<20 and t_distancia_avg[i+1]> t_distancia_avg[i+2]:
                    print("aplico 5.2")
                    max_index=t_cluster[i+2]
                else:
                    print(" aplico 5.3")
                    max_index=t_cluster[i+3]
            else:
            #    print(" aplico 6")
            #    max_index = t_cluster[i]
                if t_distancia_avg[i] >45:
                    print("aplico 6.1")
                    max_index = t_cluster[i+1]
                else:
                    print("aplico 6.2")
                    max_index = t_cluster[i]
            
            break       

    print("El índice del mayor valor que no es un valor atípico es:", max_index)
##===================================================
## Este es el fin de la optimización sin outliers
##===================================================
    
    c_df, c_idx = cluster_N_extrems_seismic (multi_df, prm_sorted_signal,max_index)
    
    ##=================================================
    ## Grafico de conglomerados de la onda original
    ##=================================================
    print(prm_sorted_signal.c)
    print("Gráfico de conglomerados de la onda original")
    plt.figure(figsize=(30,15))
    plt.scatter(t_array,y_array, c=cluster_array, cmap='rainbow')
    # Configurar las etiquetas de los ejes
    plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
    plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)
    
    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    plt.show()
    
    t = c_df.t.values
    y = c_df.y.values

    y_min =np.min(y)
    cero_count =0
    t_m = df
    cero_period = tiempo_entre_explosiones_min
    t_inicio     = []
    t_inicio_val = []
    t_final      = []
    t_final_val  = []
    t_distancia  = []
    t_cero = t[0]
    t_last =int(t_cero)
    
    last_t = t[0]
    very_last = t[0]
    count_ponts = 0
    for ji in t[1:]:
        if (ji - very_last) >= t_m*cero_period and (very_last - last_t) <= t_m:
            last_t = ji
            very_last_t = ji
        elif (ji - very_last) >= t_m*cero_period and (very_last - last_t) > t_m:

            t_inicio.append(last_t)

            t_inicio_val.append(y_array[int(last_t)])
            t_final.append(very_last)
            t_final_val.append(y_array[int(very_last)])
            last_t = ji
            if ji == t[-1]:
                t_inicio.append(last_t)
                t_inicio_val.append(y_array[int(last_t)])
                t_final.append(ji)
                t_final_val.append(y_array[int(ji)])
                break
        if ji == t[-1]:
            t_inicio.append(last_t)
            t_inicio_val.append(y_array[int(last_t)])
            t_final.append(ji)
            t_final_val.append(y_array[int(ji)])
            break
        very_last = ji
            
    plt.figure(figsize=(30,15))
    plt.scatter(t_array,y_array,c='g')
    plt.scatter(c_df.t,c_df.y, c='orange')
    plt.scatter(t_inicio,t_inicio_val,c ='r')
    plt.scatter(t_final,t_final_val, c = 'b')


    # Agregar las barras verticales para indicar el inicio y fin del evento
    for inicio, fin in zip(t_inicio, t_final):
        plt.axvline(x=inicio, color='r', linestyle='--')
        plt.axvline(x=fin, color='g', linestyle='--')

    # Configurar las etiquetas de los ejes
    plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
    plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)
    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)

    # Mostrar el plot
    plt.show()
    

    ##################################################################################
    ##
    ## Aquí  inicia el análisis de intervalos
    ##
    ##################################################################################
    t_times = []
    for i in range(len(t_inicio[1:])):
        t_times.append(t_inicio[i])
        t_times.append(t_final[i])

    tr_times = []
    tr_time_ini =[]
    tr_time_fin =[]
    explo_count = 0
    explode = []
    
    for v in tr_EHZ.times("utcdatetime")  :
        tr_times.append(v)

    for i in range(len(t_inicio)):#[1:])):
        tini = t_inicio[i]
        tfin = t_final [i]
        if tini < 0:
            tini = 0
        if tfin >= len(tr_times):
            tfin = len(tr_times)-1
        tr_time_ini.append(tr_times[int(tini)])
        tr_time_fin.append(tr_times[int(tfin)])
        explo_count +=1
        explode.append(explo_count-1)
     
    explode_2d = np.expand_dims(explode, axis=1)
    tr_time_ini_2d = np.expand_dims(tr_time_ini, axis=1)
    tr_time_fin_2d = np.expand_dims(tr_time_fin, axis=1)

    the_times = np.concatenate((explode_2d,tr_time_ini_2d,tr_time_fin_2d),axis=1)
    EHZ_times = pd.DataFrame(the_times, columns = ['explode','time_start','time_end'] )
    it = 0
        
    for ix in range(len(EHZ_times)):
        time_INI  = EHZ_times.iloc[ix,1]    
        time_FIN  = EHZ_times.iloc[ix,2]
        print (time_INI,time_FIN,time_FIN-time_INI)
        seismic_inicio.append(time_INI)
        seismic_fin.append(time_FIN)
       

    
            
    if tr_time_ini != [] and tr_time_fin !=[]:
        for dti in tr_time_ini:
            det_tr_inicio.append(dti)
        for dtf in tr_time_fin:
            det_tr_fin.append(dtf)

        
det_tr_seismic_inicio = det_tr_inicio
det_tr_seismic_fin    = det_tr_fin

# Procesamiento de señal acústica registrada por el sismómetro

In [ ]:
pick = UTC('2022-09-17T0:00:00')

st__HDF = read("GI.FG12.02.BDF.D.2022.260.mseed")
st__EHZ = read("GI.FG12.00.BHZ.D.2022.260.mseed")

tr_HDF = st__HDF[0]
tr_EHZ = st__EHZ[0]

tr_tr = range(0,tr_EHZ.stats.npts,pick_up)
l_t_i = tr_EHZ.stats.starttime


times = np.arange(tr_EHZ.times("utcdatetime")[0], tr_EHZ.times("utcdatetime")[-1], pick_up)
ii = 1
l_t_i = times[0]
detecciones = []
det_inicio = []
det_final = []
det_tr_inicio =[]
det_tr_fin = []
for t in times[1:]:
    print(" ") 
    print(" ") 
    print(" ") 
    print("==========================")
    print(ii,t)
    print(" ") 
    print(" ") 
    ii +=1
    t_i = t
    #print(t_i)
    #print(l_t_i, t_i)
    st_HDF_trimmed = st__HDF.copy()
    st_EHZ_trimmed = st__EHZ.copy()
    st_EHZ = st_EHZ_trimmed.trim(l_t_i, t_i)
    st_HDF = st_HDF_trimmed.trim(l_t_i, t_i)
    st_HDF = st_HDF.filter('bandpass', freqmin= fmin_HDF, freqmax=fmax_HDF, corners=2, zerophase=True)
    st_EHZ = st_EHZ.filter('bandpass', freqmin= fmin_EHZ, freqmax=fmax_EHZ, corners=2, zerophase=True)
    tr_HDF = st_HDF[0]
    tr_EHZ = st_EHZ[0]
    l_t_i =  t_i

    serie  = tr_EHZ.data 
    signal = pd.Series(serie)
    ss, er, erv,s_mx,s_mn = suavizado_señal(signal,51)
    dist = np.diff(er)
    extrems = encontrar_extremos_relativos(ss)
    t = np.arange(len(extrems))

    plt.figure(figsize=(35,15))
    plt.plot(signal, color='#1E90FF', label='Señal original', zorder=0)
    plt.scatter(t,extrems, c='#32CD32',label="valores extremos", zorder=2)
    plt.plot(ss, color='#FF4500', label= "onda suavizada", zorder=1)
    plt.legend(fontsize=25)
    
    # Configurar las etiquetas de los ejes
    plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
    plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)
    
    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    plt.show()
    
    ##########################################
    ##########################################
    ##                                      ##
    ##           Clústers                   ##
    ##                                      ##
    ##########################################
    ##########################################

    t = np.arange(len(tr_EHZ.data))
    y_array      = []
    y_sv_array   = []
    x_array      = []
    t_array      = []
    for r in extrems.data:
        x_array.append([r])

    for s in tr_EHZ.data:
        y_array.append([s])
    for s in ss:
        y_sv_array.append([s])

    for u in t:
        t_array.append([u])

    n_clusters = n_Clusters
    kmeans = KMeans(n_Clusters)
    
    cluster_array     = calculo_clusters_signal(x_array,y_array)
    x_multi           = np.concatenate((t_array,x_array,y_array,y_sv_array, cluster_array),axis=1)
    multi_df          = pd.DataFrame(x_multi, columns = ['t','x','y','y_sv','c'] )
    prm_sorted_signal = estadisticas_conglomerados(multi_df)
    
##===================================================
## Optimización de detecciones descontando outliers
##===================================================
    k = int((n_clusters - 1) / 2)

    # Generar los valores de la secuencia en un rango de 1 a n
    sequence = list(range(1, k + 1))

    # Ordenar la secuencia de mayor a menor
    sequence.sort(reverse=True)

    t_distancia_avg = []
    t_conteo        = []
    t_cluster       = []


    # Iterar en un ciclo for
    for x_clusters in sequence:
        t_inicio        = []
        t_inicio_val    = []
        t_final         = []
        t_final_val     = []
        t_distancia     = []
        
        c_df, c_idx = cluster_N_extrems (multi_df, prm_sorted_signal,x_clusters)
        t = c_df.t.values
        y = c_df.y.values

    
        last_t = t[0]
        very_last = t[0]
        for ji in t[1:]:
            if (ji - very_last) >= t_m*cero_period:
                t_inicio.append([last_t])
                t_inicio_val.append(y_array[int(last_t)])
                t_final.append([very_last])
                t_final_val.append(y_array[int(very_last)])
                t_distancia.append(abs(last_t-very_last))

                last_t = ji
                if ji == t[-1]:
                    t_inicio.append([last_t])
                    t_inicio_val.append([y_array[int(last_t)]])
                    t_final.append([ji])
                    t_final_val.append(y_array[int(ji)])
                    t_distancia.append(abs(last_t-ji))
                    break
            if ji == t[-1]:
                #print('hay que hacer otro corte de emergencia')
                #print('los límites del otro intervalo de emergencia son', last_t, ji)
                t_inicio.append([last_t])
                t_inicio_val.append([y_array[int(last_t)]])
                t_final.append([ji])
                t_final_val.append(y_array[int(ji)])
                t_distancia.append(abs(last_t-ji))
                break
            very_last = ji
        t_conteo.append(len(t_final))
        t_cluster.append(x_clusters)
        t_distancia_avg.append(np.mean(t_distancia))
    
    
    # Calcular el percentil 90
    #PCT9 = math.ceil(np.percentile(t_conteo,90))
    #Q1 = np.percentile(t_conteo,25)
    #Q3 = np.percentile(t_conteo,75)
    #IQR = Q3-Q1
    outliers = []
    no_hay_conteos = False
    t_conteos_max = np.argmax(t_conteo)
    
    for i in range(len(t_conteo)):
        if all(x == t_conteo[0] for x in t_conteo):
            #print("aplico 1")
            max_index = round(len(t_conteo)/2, 0)
            break

        if i >=1:
            if t_conteo[i]>1 and t_conteo[i-1]==1:
                if t_cluster[i] == t_cluster[t_conteos_max] and t_conteo[i-1] ==1 and t_conteo[1-2]== 1 and i < len(t_conteo)-3:
                    #print("aplico 2")
                    max_index = t_cluster[i+1]
                    break
                    
                if t_cluster[i] == t_cluster[t_conteos_max] and   t_conteo[t_conteos_max] <=20 and (t_conteo[t_conteos_max]-t_conteo[t_conteos_max+1])>5:
                    #print("aplico 3")
                    max_index = t_cluster[i]
                    break
                    
                if i == len(t_conteo) - 1:
                    #print("aplico 4")
                    max_index = t_cluster[i-1]
                    break
                elif i < len(t_conteo)-3:
                    if t_cluster[i+1] == t_cluster[t_conteos_max] and t_conteo[t_conteos_max] <=20:
                        #print("aplico 5")
                        max_index = t_cluster[i+1]
                    else:
                        #print("aplico 6")
                        max_index = t_cluster[i+2]
                    break            
        


##===================================================
## Este es el fin de la iptimización sin outliers
##===================================================
    c_df, c_idx = cluster_N_extrems (multi_df, prm_sorted_signal,max_index)
    
    ##=================================================
    ## Grafico de conglomerados de la onda original
    ##=================================================
    print(prm_sorted_signal.c)
    print("Gráfico de conglomerados de la onda original")
    plt.figure(figsize=(30,15))
    plt.scatter(t_array,y_array, c=cluster_array, cmap='rainbow')
    
    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)

    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    
    # Configurar las etiquetas de los ejes
    plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
    plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)
    plt.show()
    
    t = c_df.t.values
    y = c_df.y.values

    y_min =np.min(y)
    cero_count =0
    t_m = df
    cero_period = tiempo_entre_explosiones_min
    t_inicio     = []
    t_inicio_val = []
    t_final      = []
    t_final_val  = []
    t_distancia  = []
    t_cero = t[0]
    t_last =int(t_cero)
    
    last_t = t[0]
    very_last = t[0]
    for ji in t[1:]:
        if (ji - very_last) >= t_m*cero_period:
            t_inicio.append([last_t])
            t_inicio_val.append(y_array[int(last_t)])
            t_final.append([very_last])
            t_final_val.append(y_array[int(very_last)])
            last_t = ji
            if ji == t[-1]:
                t_inicio.append([last_t])
                t_inicio_val.append([y_array[int(last_t)]])
                t_final.append([ji])
                t_final_val.append(y_array[int(ji)])
                break
        if ji == t[-1]:
            t_inicio.append([last_t])
            t_inicio_val.append([y_array[int(last_t)]])
            t_final.append([ji])
            t_final_val.append(y_array[int(ji)])
            break
        very_last = ji
                
    tremores= False
    for j in np.arange(len(t_final)):
        t_distancia.append((t_final[j][0]-t_inicio[j][0])/t_m/60)
        if (t_final[j][0]-t_inicio[j][0])/t_m/60 >1.5:
            tremores = True


    print("Distancias",t_distancia)
    print("inicio",t_inicio)
    print("final",t_final)  
 
    plt.figure(figsize=(35,15))
    plt.scatter(t_array,y_array,c='g')
    plt.scatter(c_df.t,c_df.y, c='orange')

    # Agregar las barras verticales para indicar el inicio y fin del evento
    for inicio, fin in zip(t_inicio, t_final):
        plt.axvline(x=inicio, color='r', linestyle='--')
        plt.axvline(x=fin, color='g', linestyle='--')

    # Configurar las etiquetas de los ejes
    plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
    plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)

    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)

    # Mostrar el plot
    plt.show()
    

    ##################################################################################
    ##
    ## Aquí  inicia el análisis de correlación cruzada
    ##
    ##################################################################################
    t_times = []
    for i in range(len(t_inicio[1:])):
        t_times.append(t_inicio[i])
        t_times.append(t_final[i])

    stream = st_EHZ
    stream += st_HDF
    templates = []
    template_names = []
    tr_times = []
    tr_time_ini =[]
    tr_time_fin =[]
    explo_count = 0
    explode = []
    
    for v in tr_EHZ.times("utcdatetime")  :
        tr_times.append(v)
    
    for i in range(len(t_inicio)):#[1:])):
        tini = t_inicio[i][0]
        tfin = t_final [i][0]
        if tini < 0:
            tini = 0
        if tfin >= len(tr_times):
            tfin = len(tr_times)-1
        tr_time_ini.append(tr_times[int(tini)])
        tr_time_fin.append(tr_times[int(tfin)])
        explo_count +=1
        explode.append([explo_count-1])
     
    tr_time_ini_2d = np.expand_dims(tr_time_ini, axis=1)
    tr_time_fin_2d = np.expand_dims(tr_time_fin, axis=1)

    the_times = np.concatenate((explode,tr_time_ini_2d,tr_time_fin_2d),axis=1)
    EHZ_times = pd.DataFrame(the_times, columns = ['explode','time_start','time_end'] )
    it = 0
    
    for ix in range(len(EHZ_times)):
        time_INI  = EHZ_times.iloc[ix,1]    
        time_FIN  = EHZ_times.iloc[ix,2]
        print (time_INI,time_FIN,time_FIN-time_INI)
            
    if tr_time_ini != [] and tr_time_fin !=[]:
        for dti in tr_time_ini:
            det_tr_inicio.append(dti)
        for dtf in tr_time_fin:
            det_tr_fin.append(dtf)

det_tr_EHZ_inicio = det_tr_inicio
det_tr_EHZ_fin    = det_tr_fin

# Procesamiento de señal acústica

In [ ]:
pick = UTC('2022-09-17T0:00:00')

tiempo_entre_explosiones_min=10

st__HDF = read("GI.FG12.02.BDF.D.2022.260.mseed")
st__EHZ = read("GI.FG12.00.BHZ.D.2022.260.mseed")

tr_HDF = st__HDF[0]
tr_EHZ = st__EHZ[0]

time_relative = tr_HDF.stats.npts/tr_HDF.stats.sampling_rate
tr_tr = range(0,tr_HDF.stats.npts,pick_up)
l_t_i = tr_HDF.stats.starttime


times = np.arange(tr_HDF.times("utcdatetime")[0], tr_HDF.times("utcdatetime")[-1], pick_up)
ii = 1
l_t_i = times[0]
detecciones = []
det_inicio = []
det_final = []
det_tr_inicio =[]
det_tr_fin = []
t_m = df
cero_period = tiempo_entre_explosiones_min
    
for t in times[1:]:
    print(" ") 
    print(" ") 
    print(" ") 
    print("==========================")
    print(ii,t)
    print(" ") 
    print(" ") 
    ii +=1
    t_i = t

    st_HDF_trimmed = st__HDF.copy()
    st_EHZ_trimmed = st__EHZ.copy()
    st_EHZ = st_EHZ_trimmed.trim(l_t_i, t_i)
    st_HDF = st_HDF_trimmed.trim(l_t_i, t_i)
    st_HDF = st_HDF.filter('bandpass', freqmin= fmin_HDF, freqmax=fmax_HDF, corners=2, zerophase=True)
    st_EHZ = st_EHZ.filter('bandpass', freqmin= fmin_EHZ, freqmax=fmax_EHZ, corners=2, zerophase=True)
    tr_HDF = st_HDF[0]
    tr_EHZ = st_EHZ[0]
    l_t_i =  t_i

    serie  = tr_HDF.data 
    signal = pd.Series(serie)
    ss, er, erv,s_mx,s_mn = suavizado_señal(signal,51)

    plt.figure(figsize=(25,10))
    plt.plot(signal, label='Señal original')
    plt.plot(ss, label='Señal suavizada')
    plt.legend(fontsize=25)
    
    # Configurar las etiquetas de los ejes
    plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
    plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)
    
    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    plt.show()

    dist = np.diff(er)

    extrems = encontrar_extremos_relativos(ss)
    t = np.arange(len(extrems))
    plt.figure(figsize=(35,15))
    plt.plot(signal, color='#1E90FF', label='Señal original', zorder=0)
    plt.scatter(t,extrems, c='#32CD32',label="valores extremos", zorder=2)
    plt.plot(ss, color='#FF4500', label= "onda suavizada", zorder=1)
    plt.legend(fontsize=25)
    
    # Configurar las etiquetas de los ejes
    plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
    plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)
    
    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    plt.show()
    
    ##########################################
    ##########################################
    ##                                      ##
    ##           Clústers                   ##
    ##                                      ##
    ##########################################
    ##########################################

    t = np.arange(len(tr_HDF.data))
    y_array      = []
    y_sv_array   = []
    x_array      = []
    t_array      = []
    for r in extrems.data:
        x_array.append([r])

    for s in tr_HDF.data:
        y_array.append([s])
    for s in ss:
        y_sv_array.append([s])

    for u in t:
        t_array.append([u])

    n_clusters = n_Clusters
    kmeans = KMeans(n_Clusters)
    
    cluster_array     = calculo_clusters_signal(x_array,y_array)
    x_multi           = np.concatenate((t_array,x_array,y_array,y_sv_array, cluster_array),axis=1)
    multi_df          = pd.DataFrame(x_multi, columns = ['t','x','y','y_sv','c'] )
    prm_sorted_signal = estadisticas_conglomerados(multi_df)
    
    c_c = prm_sorted_signal.c[7]
    
##===================================================
## Optimización de detecciones descontando outliers
##===================================================
    k = int((n_clusters - 1) / 2)

    # Generar los valores de la secuencia en un rango de 1 a n
    sequence = list(range(1, k + 1))

    # Ordenar la secuencia de mayor a menor
    sequence.sort(reverse=True)

    t_distancia_avg = []
    t_conteo        = []
    t_cluster       = []

    # Iterar en un ciclo for
    for x_clusters in sequence:
        t_inicio        = []
        t_inicio_val    = []
        t_final         = []
        t_final_val     = []
        t_distancia     = []
        c_df, c_idx = cluster_N_extrems (multi_df, prm_sorted_signal,x_clusters)
        t = c_df.t.values
        y = c_df.y.values

    
        last_t = t[0]
        very_last = t[0]
        for ji in t[1:]:
            if (ji - very_last) >= t_m*cero_period:
                t_inicio.append([last_t])
                t_inicio_val.append(y_array[int(last_t)])
                t_final.append([very_last])
                t_final_val.append(y_array[int(very_last)])
                t_distancia.append(abs(last_t-very_last))

                last_t = ji
                if ji == t[-1]:
                    t_inicio.append([last_t])
                    t_inicio_val.append([y_array[int(last_t)]])
                    t_final.append([ji])
                    t_final_val.append(y_array[int(ji)])
                    t_distancia.append(abs(last_t-ji))
                    break
            if ji == t[-1]:
                t_inicio.append([last_t])
                t_inicio_val.append([y_array[int(last_t)]])
                t_final.append([ji])
                t_final_val.append(y_array[int(ji)])
                t_distancia.append(abs(last_t-ji))
                break
            very_last = ji
        t_conteo.append(len(t_final))
        t_cluster.append(x_clusters)
        t_distancia_avg.append(np.mean(t_distancia))
        print(x_clusters, len(t_final),np.mean(t_distancia) )
    
    
    # Calcular el percentil 90
    PCT9 = math.ceil(np.percentile(t_conteo,90))
    Q1 = np.percentile(t_conteo,25)
    Q3 = np.percentile(t_conteo,75)
    IQR = Q3-Q1
    outliers = []
    no_hay_conteos = False
    t_conteos_max = np.argmax(t_conteo)
    for i in range(len(t_conteo)):
        if all(x == t_conteo[0] for x in t_conteo):
            #print("aplico 1")
            max_index = round(len(t_conteo)/2, 0)
            break  
        if i >=1:
            if t_conteo[i]>1 and t_conteo[i-1]==1:
                
                if t_conteos_max < len(t_conteo)-1:
                    if  t_cluster[i] == t_cluster[t_conteos_max] and   t_conteo[t_conteos_max] <=20 and (t_conteo[t_conteos_max]-t_conteo[t_conteos_max+1])>5:
               #         print("aplico 2")
                        if  i> 1:
               #             print("aplico 2.1")
                            max_index = t_cluster[i]
                        else:
               #             print("aplico 2.2")
                            max_index = t_cluster[i+1]
                        break
                    
                if i == len(t_conteo) - 1:
               #     print("aplico 3")
                    max_index = t_cluster[i-1]
                    break
        if (i < len(t_conteo)-3 and i >=2
            and t_conteo[i-2] ==1
            and t_conteo[i-1] >1 
           ):
           # print("aplico 4")
            max_index = t_cluster[i]
            if t_conteo[max_index] >20:
           #     print("aplico 5")
                max_index = t_cluster[i+1]
            elif t_distancia_avg[max_index] > 45 and t_distancia_avg[max_index] > t_distancia_avg[max_index+1]:
            #    print("aplico 6")
                max_index=t_cluster[i+1]
            else:
             #   print("aplico 7")
                max_index = t_cluster[i]
            
            break 

    print("El índice del mayor valor que no es un valor atípico es:", max_index)
##===================================================
## Este es el fin de la iptimización sin outliers
##===================================================

    c_df, c_idx = cluster_N_extrems (multi_df, prm_sorted_signal,max_index)

    ##=================================================
    ## Grafico de conglomerados de la onda original
    ##=================================================
    print("Gráfico de conglomerados de la onda original")
    plt.figure(figsize=(30,15))
    plt.scatter(t_array,y_array, c=cluster_array, cmap='rainbow')

    # Configurar las etiquetas de los ejes
    plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
    plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)
    
    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    plt.show()
    
    t = c_df.t.values
    y = c_df.y.values

    y_min =np.min(y)
    cero_count =0
    t_m = df
    cero_period = tiempo_entre_explosiones_min
    t_inicio     = []
    t_inicio_val = []
    t_final      = []
    t_final_val  = []
    t_distancia  = []
    t_cero = t[0]
    t_last =int(t_cero)
    
    last_t = t[0]
    very_last = t[0]
    for ji in t[1:]:
        if (ji - very_last) >= t_m*cero_period:
            t_inicio.append([last_t])
            t_inicio_val.append(y_array[int(last_t)])
            t_final.append([very_last])
            t_final_val.append(y_array[int(very_last)])
            last_t = ji
            if ji == t[-1]:
                # hay que hacer un corte de emergencia
                # los límites del intervalo de emergencia son', ji, ji
                t_inicio.append([last_t])
                t_inicio_val.append([y_array[int(last_t)]])
                t_final.append([ji])
                t_final_val.append(y_array[int(ji)])
                break

        if ji == t[-1]:
            t_inicio.append([last_t])
            t_inicio_val.append([y_array[int(last_t)]])
            t_final.append([ji])
            t_final_val.append(y_array[int(ji)])
            break
        very_last = ji
   
    tremores= False
    for j in np.arange(len(t_final)):
        t_distancia.append((t_final[j][0]-t_inicio[j][0])/t_m/60)
        if (t_final[j][0]-t_inicio[j][0])/t_m/60 >1.5:
            tremores = True


    print("Distancias",t_distancia)
    print("inicio",t_inicio)
    print("final",t_final)  
    
    plt.figure(figsize=(25,10))
    plt.scatter(t_array,y_array,c='g')
    plt.scatter(c_df.t,c_df.y, c='orange')
    
    # Agregar las barras verticales para indicar el inicio y fin del evento
    for inicio, fin in zip(t_inicio, t_final):
        plt.axvline(x=inicio, color='r', linestyle='--')
        plt.axvline(x=fin, color='g', linestyle='--')

    # Configurar las etiquetas de los ejes
    plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
    plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)

    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    
    # Mostrar el plot
    plt.show()
    
    t_times = []
    for i in range(len(t_inicio[1:])):
        t_times.append(t_inicio[i])
        t_times.append(t_final[i])

    stream = st_EHZ
    stream += st_HDF
    templates = []
    template_names = []
    tr_times = []
    tr_time_ini =[]
    tr_time_fin =[]
    explo_count = 0
    explode = []
    for v in tr_HDF.times("utcdatetime")  :
        tr_times.append(v)
    
    for i in range(len(t_inicio)):#[1:])):
        tini = t_inicio[i][0]
        tfin = t_final [i][0]
        if tini < 0:
            tini = 0
        if tfin >= len(tr_times):
            tfin = len(tr_times)-1
        tr_time_ini.append(tr_times[int(tini)])
        tr_time_fin.append(tr_times[int(tfin)])
        explo_count +=1
        explode.append([explo_count-1])
     
    tr_time_ini_2d = np.expand_dims(tr_time_ini, axis=1)
    tr_time_fin_2d = np.expand_dims(tr_time_fin, axis=1)

    the_times = np.concatenate((explode,tr_time_ini_2d,tr_time_fin_2d),axis=1)
    HDF_times = pd.DataFrame(the_times, columns = ['explode','time_start','time_end'] )
    
    it = 0
    
    for ix in range(len(HDF_times)):
        time_INI  = HDF_times.iloc[ix,1]    
        time_FIN  = HDF_times.iloc[ix,2]
        print (time_INI,time_FIN,time_FIN-time_INI)
        
    if tr_time_ini != [] and tr_time_fin !=[]:
        for dti in tr_time_ini:
            det_tr_inicio.append(dti)
        for dtf in tr_time_fin:
            det_tr_fin.append(dtf)


det_tr_HDF_inicio = det_tr_inicio
det_tr_HDF_fin = det_tr_fin            


In [ ]:
df_eventos_sismo_acusticos=pd.concat([pd.Series(det_tr_EHZ_inicio),pd.Series(det_tr_EHZ_fin),pd.Series(det_tr_EHZ_fin)-pd.Series(det_tr_EHZ_inicio)], axis=1)
df_eventos_sismo_acusticos.columns=['event_ini','event_fin','diff']
df_eventos_sismo_acusticos.to_excel("df_eventos_sismo_acusticos.xlsx")
df_eventos_acusticos=pd.concat([pd.Series(det_tr_HDF_inicio),pd.Series(det_tr_HDF_fin),pd.Series(det_tr_HDF_fin)-pd.Series(det_tr_HDF_inicio)], axis=1)
df_eventos_acusticos.columns=['event_ini','event_fin','diff']
df_eventos_acusticos.to_excel("df_eventos_acusticos.xlsx")
df_eventos_sismicos=pd.concat([pd.Series(seismic_inicio),pd.Series(seismic_fin),pd.Series(seismic_fin)-pd.Series(seismic_inicio)], axis=1)
df_eventos_sismicos.columns=['event_ini','event_fin','diff']
df_eventos_sismicos.to_excel("df_eventos_sismicos.xlsx")

# Reacondicionamiento de intervalos

In [ ]:
def ajustar_intervalos(inicio_intervalo, fin_intervalo):
    n = len(inicio_intervalo)
    i = 0
    
    while i < n - 1:
        if ((inicio_intervalo[i + 1] - fin_intervalo[i]) < 1):
            
            # Si la diferencia es 1, entonces se deben unir los intervalos
            fin_intervalo[i] = fin_intervalo[i + 1]
            # Eliminamos el inicio y fin del intervalo siguiente
            del inicio_intervalo[i + 1]
            del fin_intervalo[i + 1]
            n -= 1
        else:
            i += 1
        
    return inicio_intervalo, fin_intervalo

seismic_inicio_ajustado, seismic_fin_ajustado = ajustar_intervalos(seismic_inicio, seismic_fin)
det_tr_EHZ_inicio_ajustado, det_tr_EHZ_fin_ajustado = ajustar_intervalos(det_tr_EHZ_inicio, det_tr_EHZ_fin)
det_tr_HDF_inicio_ajustado, det_tr_HDF_fin_ajustado = ajustar_intervalos(det_tr_HDF_inicio, det_tr_HDF_fin)

print(len(det_tr_seismic_inicio),len(seismic_inicio_ajustado))
print(len(det_tr_EHZ_inicio),len(det_tr_EHZ_inicio_ajustado))
print(len(det_tr_HDF_inicio),len(det_tr_HDF_inicio_ajustado))

# Preprocesamiento de detecciones de explosiones

In [ ]:
hdf_tiempos_ini = []
ehz_tiempos_ini = []
hdf_tiempos_fin = []
ehz_tiempos_fin = []
diff_hdf = []
diff_ehz =[]
diff_ini = []
diff_fin =[]
j = 0
i = 0
k =1

el_tiempo_t = UTC("2022-09-17T01:41:26.380000Z")
el_tiempo_t2 = UTC("2022-09-17T01:41:26.460000Z")
lo_encontré= False
while i < len(det_tr_HDF_inicio):
    j=0
    while j < len(det_tr_EHZ_inicio):
        time_delta_seconds = abs(det_tr_HDF_inicio[i] - det_tr_EHZ_inicio[j]) #/ pd.Timedelta(seconds=1)

        if (det_tr_HDF_inicio[i] <= det_tr_EHZ_inicio[i] 
            and det_tr_HDF_fin[i]>= det_tr_EHZ_fin[i]
            and time_delta_seconds <=dif_ini_delay):
            print('hallazgo:',k,'en i',i,' j:',j,'ehz:', det_tr_HDF_inicio[i], 'hdf: ',det_tr_EHZ_inicio[j], 'time_delta:', time_delta_seconds)
            time_delta_seconds_fin = abs(det_tr_HDF_fin[i] - det_tr_EHZ_fin[j]) #/ pd.Timedelta(seconds=1)
            print('  intervalo_HDF fin', det_tr_HDF_fin[i], 'inicio', det_tr_HDF_inicio[i], det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])
            print('  intervalo_EHZ fin', det_tr_EHZ_fin[j], 'inicio', det_tr_EHZ_inicio[j], det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j])
            print('  intervalos EHZ',det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j],'HDF',det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])
            print('diferencia      ',  abs((det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j]) -(det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])))
            hdf_tiempos_ini.append(det_tr_HDF_inicio[i])
            ehz_tiempos_ini.append(det_tr_EHZ_inicio[j])
            hdf_tiempos_fin.append(det_tr_HDF_fin[i])
            ehz_tiempos_fin.append(det_tr_EHZ_fin[j])
            diff_hdf.append(det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])
            diff_ehz.append(det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j])
            diff_fin.append(((det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j]) -(det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])))
            diff_ini.append((det_tr_EHZ_inicio[j]- det_tr_HDF_inicio[i]))
            j+=1
            k+=1
            break
        if (det_tr_EHZ_inicio[i] <= det_tr_HDF_inicio[i]
            and det_tr_EHZ_fin[i]>= det_tr_HDF_fin[i]
            and time_delta_seconds <=dif_ini_delay):
            print('hallazgo:',k,'en i',i,' j:',j,'ehz:', det_tr_HDF_inicio[i], 'hdf: ',det_tr_EHZ_inicio[j], 'time_delta:', time_delta_seconds)
            time_delta_seconds_fin = abs(det_tr_HDF_fin[i] - det_tr_EHZ_fin[j]) #/ pd.Timedelta(seconds=1)
            print('  intervalo_HDF fin', det_tr_HDF_fin[i], 'inicio', det_tr_HDF_inicio[i], det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])
            print('  intervalo_EHZ fin', det_tr_EHZ_fin[j], 'inicio', det_tr_EHZ_inicio[j], det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j])
            print('  intervalos EHZ',det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j],'HDF',det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])
            print('diferencia      ',  abs((det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j]) -(det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])))
            hdf_tiempos_ini.append(det_tr_HDF_inicio[i])
            ehz_tiempos_ini.append(det_tr_EHZ_inicio[j])
            hdf_tiempos_fin.append(det_tr_HDF_fin[i])
            ehz_tiempos_fin.append(det_tr_EHZ_fin[j])
            diff_hdf.append(det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])
            diff_ehz.append(det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j])
            diff_fin.append((det_tr_EHZ_fin[j]   - det_tr_HDF_fin[i]))
            diff_ini.append((det_tr_EHZ_inicio[j]- det_tr_HDF_inicio[i]))
            j+=1
            k+=1
            break    
        if  (time_delta_seconds <=dif_ini_delay 
            ): 
            print('hallazgo:',k,'en i',i,' j:',j,'ehz:', det_tr_HDF_inicio[i], 'hdf: ',det_tr_EHZ_inicio[j], 'time_delta:', time_delta_seconds)
            time_delta_seconds_fin = abs(det_tr_HDF_fin[i] - det_tr_EHZ_fin[j]) #/ pd.Timedelta(seconds=1)
            print('  intervalo_HDF fin', det_tr_HDF_fin[i], 'inicio', det_tr_HDF_inicio[i], det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])
            print('  intervalo_EHZ fin', det_tr_EHZ_fin[j], 'inicio', det_tr_EHZ_inicio[j], det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j])
            print('  intervalos EHZ',det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j],'HDF',det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])
            print('diferencia      ',  abs((det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j]) -(det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])))
                         
            hdf_tiempos_ini.append(det_tr_HDF_inicio[i])
            ehz_tiempos_ini.append(det_tr_EHZ_inicio[j])
            hdf_tiempos_fin.append(det_tr_HDF_fin[i])
            ehz_tiempos_fin.append(det_tr_EHZ_fin[j])
            diff_hdf.append(det_tr_HDF_fin[i] - det_tr_HDF_inicio[i])
            diff_ehz.append(det_tr_EHZ_fin[j] - det_tr_EHZ_inicio[j])
            diff_fin.append((det_tr_EHZ_fin[j]   - det_tr_HDF_fin[i]   ))
            diff_ini.append((det_tr_EHZ_inicio[j]- det_tr_HDF_inicio[i]))
            j+=1
            k+=1
            break
        j+=1
    i+=1
df_intervalos = pd.concat([pd.Series(hdf_tiempos_ini), pd.Series(hdf_tiempos_fin), pd.Series(ehz_tiempos_ini), pd.Series(ehz_tiempos_fin), pd.Series(diff_hdf), pd.Series(diff_ehz), pd.Series(diff_ini), pd.Series(diff_fin)], axis=1)
df_intervalos.columns =['hdf_ini','hdf_fin','ehz_ini','ehz_fin','diff_hdf','diff_ehz','diff_ini','diff_fin']
df_intervalos.diff_fin.describe()
df_intervalos.to_excel("df_intervalos.xlsx")

# Cálculo de intervalos

In [ ]:
if df_intervalos.hdf_ini.iloc[0] <= df_intervalos.ehz_ini.iloc[0]:
    l_t_i = df_intervalos.hdf_ini.iloc[0]
else:
    l_t_i = df_intervalos.ehz_ini.iloc[0]
j =0
init()
t_i = l_t_i+86400
det_tr_ini =[]
det_tr_fin = []
l_t_i 
print(l_t_i,t_i)
ini_hdf = df_intervalos['hdf_ini']
num = 1
while j <(len(df_intervalos)):
    if (df_intervalos.hdf_ini.iloc[j] >= l_t_i and df_intervalos.hdf_ini.iloc[j] <= t_i and 
        df_intervalos.hdf_fin.iloc[j] >= l_t_i and df_intervalos.hdf_fin.iloc[j] <= t_i and
        df_intervalos.ehz_ini.iloc[j] >= l_t_i and df_intervalos.ehz_ini.iloc[j] <= t_i and 
        df_intervalos.ehz_fin.iloc[j] >= l_t_i and df_intervalos.ehz_fin.iloc[j] <= t_i):
            print('intervalo',num)
            num +=1
            print('hallazgo HDF',df_intervalos.hdf_ini.iloc[j])
            print('hallazgo EHZ',df_intervalos.ehz_ini.iloc[j])
            
            if df_intervalos.diff_ini.iloc[j] >= 0:
                print("--------------  se toma el tiempo HDF_ini")
                det_tr_ini.append(df_intervalos.hdf_ini.iloc[j])
            elif df_intervalos.diff_ini.iloc[j] < 0:
                print('--------------  se toma el tiempo EHZ_ini') 
                det_tr_ini.append(df_intervalos.ehz_ini.iloc[j])

            if df_intervalos.diff_fin.iloc[j]   < 0: 
                print("                se toma el tiempo HDF fin")   
                det_tr_fin.append(df_intervalos.hdf_fin.iloc[j])
            elif df_intervalos.diff_fin.iloc[j] >= 0:
                print("                se toma el tiempo EHZ fin")
                det_tr_fin.append(df_intervalos.ehz_fin.iloc[j])                
    j+=1

df_eventos_filtrados = pd.concat([pd.Series(det_tr_ini), pd.Series(det_tr_fin),pd.Series(det_tr_fin)-pd.Series(det_tr_ini)], axis=1)
df_eventos_filtrados.columns =['event_ini','event_fin','diff']
df_eventos_filtrados.to_excel("df_eventos_filtrados.xlsx")

# Calculo la correlación cruzada entre las ondas infrasónicas

In [ ]:
from scipy import signal
st__HDF = read("GI.FG12.02.BDF.D.2022.260.mseed")
st__EHZ = read("GI.FG12.00.BHZ.D.2022.260.mseed")
st_HDF   = st__HDF.filter('bandpass', freqmin= fmin_HDF, freqmax=fmax_HDF, corners=2, zerophase=True)
st_EHZ = st__EHZ.filter('bandpass', freqmin= fmin_EHZ, freqmax=fmax_EHZ, corners=2, zerophase=True)
tr_HDF = st_HDF[0]
tr_EHZ = st_EHZ[0]
stream = st_HDF
stream += st_EHZ
detecciones_event_ini =[]
detecciones_event_fin =[]
detecciones_pascales_ehz = []
detecciones_pascales_hdf = []
detecciones_kurtosis = []
detecciones_densidad = []
detecciones_sesgo = []
detecciones_diff_max = []
detecciones_eventos_i =[]

df_eventos_filtrados = pd.read_excel("df_eventos_filtrados.xlsx")
for i, eventos_row in df_eventos_filtrados.iterrows():
    event__ini = UTC(eventos_row.event_ini)
    event__fin = UTC(eventos_row.event_fin)
    #pascal_multiplicador
    if math.isnan(event__fin):
        template_test = stream.slice(event__ini-15,event__ini+45)
    else:    
        template_test = stream.slice(event__ini,event__fin)
    test_ehz = template_test[1]
    test_hdf = template_test[0]
    test_signal_ehz = test_ehz.data
    test_signal_hdf = test_hdf.data
    
    # Calcula la correlación cruzada entre test_signal_ehz y test_signal_hdf
    corr = signal.correlate(test_signal_ehz, test_signal_hdf, mode='same', method='fft')

    # Normaliza la correlación cruzada
    corr /= np.max(np.abs(corr))    
    
    # Encuentra el índice del máximo de la correlación cruzada
    max_index = np.argmax(np.abs(corr))

    # Calcula el desfase entre las dos señales
    dt = max_index - len(test_signal_ehz) + 1
    dt /= t_m  # t_m es la frecuencia de muestreo de las señales
    
    # Calcula la similitud entre las dos señales
    similitud = np.max(np.abs(corr))
    
    # calcular la densidad de la distribución de la correlación cruzada
    try:
        density = gaussian_kde(corr)
    except:
        continue
        
    # calcular la kurtosis de la distribución de la correlación cruzada
    kurt = stats.kurtosis(corr)
    skewness = stats.skew(corr)

    # Calcular los percentiles
    p25 = np.percentile(corr, 25)  # Percentil 25
    p50 = np.percentile(corr, 50)  # Percentil 50 (mediana)
    p75 = np.percentile(corr, 75)  # Percentil 75
    
    # Obtener el índice del valor máximo
    idx_max_test_hdf = np.argmax(test_hdf)
    idx_max_test_ehz = np.argmax(test_ehz)
    idx_max_corr = np.argmax(corr)

    print( i,"ini", eventos_row.event_ini,"fin", eventos_row.event_fin, "kurt",round(kurt,2),"density",round(density(-0.5)[0],3), "sesgo", round(skewness,3),"diff",abs(idx_max_test_hdf-idx_max_test_ehz)/t_m)
    indices = np.where(np.array(corr) > cross_treshold)[0]
    
    if (len(indices) >0 and (kurt>3.6 or kurt <0) and density(-0.5) <0.135 and density(0.5) <0.2 ):       #2.74117647: #1.5:  valor dénsiti 0.135#similitud > 0.65 and np.abs(dt) < 0.06: 
        print("Pascales máximos EHZ", np.max(test_signal_ehz)*pascal_multiplicador )
        print("Pascales máximos HDF", np.max(test_signal_hdf)*pascal_multiplicador )
        print("Las señales son similares en el tiempo {:.2f} s".format(max_index/t_m))
        
        detecciones_event_ini.append(eventos_row.event_ini)
        detecciones_event_fin.append(eventos_row.event_fin)
        detecciones_pascales_ehz.append(np.max(test_signal_ehz)*pascal_multiplicador)
        detecciones_pascales_hdf.append(np.max(test_signal_hdf)*pascal_multiplicador)
        detecciones_kurtosis.append(kurt)
        detecciones_densidad.append(density(-0.5)[0])
        detecciones_sesgo.append(skewness)
        detecciones_eventos_i.append(i)
        detecciones_diff_max.append(abs(idx_max_test_hdf-idx_max_test_ehz)/t_m)
        
        #template_test.plot()
        test_hdf.plot()
        test_ehz.plot()
        plt.figure(figsize=(30,15))
        plt.plot(test_hdf,c='orange', alpha=0.5, label='Onda acústica')
        plt.plot(test_ehz, c='blue', alpha=0.5,  label='Onda sísmica')
        plt.legend(fontsize=28)
        plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
        plt.ylabel('Amplitud (cuentas digitales)', fontsize=30)

        # Ajustar el tamaño de la fuente de los ticks de los ejes
        plt.xticks(fontsize=25)
        plt.yticks(fontsize=25)
        plt.show()
        print("correlaciones")
        
        plt.figure(figsize=(30,15))
        plt.plot(corr)
        plt.xlabel('Tiempo (50 puntos por segundo)', fontsize=30)
        plt.ylabel('Coeficiente Correlación', fontsize=30)
        plt.ylabel("Densidad")
        plt.legend(fontsize=28)

        # Ajustar el tamaño de la fuente de los ticks de los ejes
        plt.xticks(fontsize=25)
        plt.yticks(fontsize=25)
        plt.show()
        # graficar la distribución de la correlación cruzada
        print("Kurtosis:", kurt)
        print("Densidad", density(-0.5))
        print("Sesgo", skewness)
        print("Similitud", similitud, "desfase ", np.abs(dt) )

        # Obtener el índice del valor máximo
        idx_max_test_hdf = np.argmax(test_hdf)
        idx_max_test_ehz = np.argmax(test_ehz)
        idx_max_corr = np.argmax(corr)

        # Imprimir los resultados
        print("Índice del valor máximo en test_hdf:", idx_max_test_hdf)
        print("Índice del valor máximo en test_ehz:", idx_max_test_ehz)
        print("Distancia entre maximos en segundos", abs(idx_max_test_hdf-idx_max_test_ehz)/t_m)
        
        # Obtener índices de los valores mayores a 0.75
        indices = np.where(np.array(corr) > 0.75)[0]

        # Imprimir los resultados
        print("Índices de los valores mayores a 0.75 en corr:", indices)
        sns.kdeplot(corr, shade=True)

        # Ajustar el tamaño de la fuente de los ticks de los ejes
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.show()
        
df_detecciones_intervalos = pd.DataFrame({'evento':detecciones_eventos_i,'event_ini': detecciones_event_ini, 'event_fin': detecciones_event_fin, 'pascales_hdf': detecciones_pascales_hdf, 'pascales_ehz':detecciones_pascales_ehz, 'kurtosis':detecciones_kurtosis, 'sesgo':detecciones_sesgo, 'densidad':detecciones_densidad, 'diff_max':detecciones_diff_max})
df_detecciones_intervalos.to_excel("de_detecciones_intervalos.xlsx")

In [ ]:
print(eventos_row.event_ini, eventos_row.event_fin)

In [ ]:
print(df_detecciones_intervalos)
print(len(detecciones_event_ini),len(detecciones_event_fin))

# Busco  la existencia de la onda sísmica

In [ ]:

event_tiempos_ini = []
event_tiempos_fin = []
seismic_tiempos_ini = []
seismic_tiempos_fin = []

j = 0
i = 0
k =1

lo_encontré= False
df_det = df_detecciones_intervalos
df_sismicos = pd.read_excel("df_eventos_sismicos.xlsx")

# Convertir las columnas event_ini y event_fin a objetos UTCDateTime
df_det['event_ini'] = df_det['event_ini'].apply(lambda x: UTC(x))
df_det['event_fin'] = df_det['event_fin'].apply(lambda x: UTC(x))

det_tr_seismic_inicio = df_sismicos['event_ini'] = df_sismicos['event_ini'].apply(lambda x: UTC(x))
det_tr_seismic_fin    = df_sismicos['event_fin'] = df_sismicos['event_fin'].apply(lambda x: UTC(x))

while i < len(df_det):

    j=0
    while j < len(det_tr_seismic_inicio):
        time_delta_seconds = abs( df_det.event_ini[i] - det_tr_seismic_inicio[j]) #/ pd.Timedelta(seconds=1)
        if  ((det_tr_seismic_inicio[j] -15<= df_det.event_ini[i] and
              det_tr_seismic_fin[j]+100 >= df_det.event_fin[i]
             )             
            ) or (i == 310 and j in [888,890]):
            print('hallazgo:',k,'en i',i,' j:',j,'ehz:', det_tr_seismic_inicio[j], 'hdf: ',df_det.event_ini[i], 'time_delta:', time_delta_seconds)
            time_delta_seconds_fin = abs(df_det.event_fin[i] - df_det.event_fin[i]) #/ pd.Timedelta(seconds=1)
            print('  intervalo_evento   ', df_det.event_fin[i], 'inicio', df_det.event_ini[i], df_det.event_fin[i] - df_det.event_ini[i])
            print('  intervalos seismic ',det_tr_seismic_fin[j],'inicio', det_tr_seismic_inicio[j],det_tr_seismic_fin[j] - det_tr_seismic_inicio[j])

            event_tiempos_ini.append(df_det.event_ini[i])
            event_tiempos_fin.append(df_det.event_fin[i])
            seismic_tiempos_ini.append(det_tr_seismic_inicio[j])
            seismic_tiempos_fin.append(det_tr_seismic_fin[j])
            
            j+=1
            k+=1
            break    
        j+=1
    i+=1
df_intervalos_2 = pd.concat([pd.Series(event_tiempos_ini), pd.Series(event_tiempos_fin),pd.Series(seismic_tiempos_ini),pd.Series(seismic_tiempos_fin)], axis=1)
df_intervalos_2.columns =['event_ini','event_fin','seismic_ini','seismic_fin']
df_intervalos_2['grand_coupling'] = df_intervalos_2['event_ini'] - df_intervalos_2['seismic_ini']

df_intervalos_2.to_excel("df_intervalos_2.xlsx")
print(df_intervalos_2)

# Calculo las correlaciones cruzadas con la onda sísmica pura

In [ ]:
st__HDF = read("GI.FG12.02.BDF.D.2022.260.mseed")
st__EHZ = read("GI.FG12.00.BHZ.D.2022.260.mseed")
st__SMC = read("GI.FG12.00.BHZ.D.2022.260.mseed")

st_HDF   = st__HDF.filter('bandpass', freqmin= fmin_HDF, freqmax=fmax_HDF, corners=2, zerophase=True)
st_EHZ = st__EHZ.filter('bandpass', freqmin= fmin_EHZ, freqmax=fmax_EHZ, corners=2, zerophase=True)
st_smc = st__SMC.filter('highpass', freq= fmin_seismic,                       corners=8, zerophase=True)
    
fmin_HDF=0.5
fmax_HDF=20
fmin_EHZ = 5#0.5 #5 #.5
fmax_EHZ = 15#5   #15 #7
fmin_seismic = 0.2
fmax_seismic = 7


tr_HDF = st_HDF[0]
tr_EHZ = st_EHZ[0]
stream = st_HDF
stream += st_EHZ
df_intervalos_2 = pd.read_excel("df_intervalos_2.xlsx")
conteo_explosiones_volcanicas = 0
indice_explosiones_volcanicas = []
grand_coupling_actualizado = []
for i, eventos_row in df_intervalos_2.iterrows():
    event__ini = UTC(eventos_row.event_ini)
    event__fin = UTC(eventos_row.event_fin)
    seismic__ini = UTC(eventos_row.seismic_ini)
    seismic__fin = UTC(eventos_row.seismic_fin)

    if math.isnan(event__fin):
        template_test = stream.slice(event__ini-15, event__ini+45)
    else:    
        template_test = stream.slice(event__ini, event__fin)
        seismic_test = st_smc.slice(seismic__ini, seismic__fin)
    
    test_ehz = template_test[1]
    test_hdf = template_test[0]
    test_smc = seismic_test[0]
    test_signal_ehz = test_ehz.data
    test_signal_hdf = test_hdf.data   
    
    print("   ")
    print("------------------------------------")
    
    print(i)
    print(eventos_row)
    
    tstmp_ehz = []
    tstmp_hdf = []
    tstmp_smc = []
    
##############################
    for v in test_ehz.times("timestamp")  :
        tstmp_ehz.append([v])
    for v in test_hdf.times("timestamp")  :
        tstmp_hdf.append([v])
    for v in test_smc.times("timestamp")  :
        tstmp_smc.append([v])
##############################

    # Obtener datos relevantes de cada fila de eventos_row
    test_ehz_data = test_ehz.data
    test_hdf_data = test_hdf.data
    test_smc_data = test_smc.data
    
    # Calcula la correlación cruzada entre test_ehz.data y test_smc.data
    corr_ehz_hdf = signal.correlate(test_ehz_data, test_hdf_data, mode='same', method='fft')

    # Calcula la correlación cruzada entre test_ehz.data y test_smc.data
    corr_ehz_smc = signal.correlate(test_ehz_data, test_smc_data, mode='same', method='fft')

    # Calcula la correlación cruzada entre test_hdf.data y test_smc.data
    corr_hdf_smc = signal.correlate(test_hdf_data, test_smc_data, mode='same', method='fft')

    # Calcula la correlación cruzada entre corr_ehz_smc y corr_hdf_smc
    corr_corr_ehz_smc_hdf_smc = signal.correlate(corr_ehz_smc, corr_hdf_smc, mode='same', method='fft')

    # Normaliza las correlaciones cruzadas
    corr_ehz_smc /= np.max(np.abs(corr_ehz_smc))
    corr_hdf_smc /= np.max(np.abs(corr_hdf_smc))
    corr_corr_ehz_smc_hdf_smc /= np.max(np.abs(corr_corr_ehz_smc_hdf_smc))

     # Calcular la densidad de la distribución de la correlación cruzada (EHZ - SMC)
    density_ehz_hdf = gaussian_kde(corr_ehz_hdf)

    # Calcular la densidad de la distribución de la correlación cruzada (EHZ - SMC)
    density_ehz_smc = gaussian_kde(corr_ehz_smc)

    # Calcular la densidad de la distribución de la correlación cruzada (HDF - SMC)
    density_hdf_smc = gaussian_kde(corr_hdf_smc)

    # Calcular la densidad de la distribución de la correlación cruzada (corr_ehz_smc - corr_hdf_smc)
    density_corr_ehz_smc_hdf_smc = gaussian_kde(corr_corr_ehz_smc_hdf_smc)

    # Calcular la kurtosis de la distribución de la correlación cruzada (EHZ - SMC)
    kurt_ehz_smc = kurtosis(corr_ehz_smc)

    # Calcular la kurtosis de la distribución de la correlación cruzada (HDF - SMC)
    kurt_hdf_smc = kurtosis(corr_hdf_smc)

    # Calcular la kurtosis de la distribución de la correlación cruzada (corr_ehz_smc - corr_hdf_smc)
    kurt_corr_ehz_smc_hdf_smc = kurtosis(corr_corr_ehz_smc_hdf_smc)

    # Calcular la asimetría (skewness) de la distribución de la correlación cruzada (EHZ - SMC)
    skewness_ehz_smc = skew(corr_ehz_smc)

    # Calcular la asimetría (skewness) de la distribución de la correlación cruzada (HDF - SMC)
    skewness_hdf_smc = skew(corr_hdf_smc)

    # Calcular la asimetría (skewness) de la distribución de la correlación cruzada (corr_ehz_smc - corr_hdf_smc)
    skewness_corr_ehz_smc_hdf_smc = skew(corr_corr_ehz_smc_hdf_smc)

    # Obtener el índice del valor máximo en las señales originales
    idx_max_test_ehz = np.argmax(test_ehz_data)
    idx_max_test_smc = np.argmax(test_smc_data)
    idx_max_test_hdf = np.argmax(test_hdf_data)

    # Obtener el índice del valor máximo en la correlación cruzada (EHZ - SMC)
    idx_max_corr_ehz_smc = np.argmax(corr_ehz_smc)

    # Obtener el índice del valor máximo en la correlación cruzada (HDF - SMC)
    idx_max_corr_hdf_smc = np.argmax(corr_hdf_smc)

    # Obtener el índice del valor máximo en la correlación cruzada (corr_ehz_smc - corr_hdf_smc)
    idx_max_corr_corr_ehz_smc_hdf_smc = np.argmax(corr_corr_ehz_smc_hdf_smc)


    grand_coupling_actualizado.append(((idx_max_test_ehz-idx_max_test_smc)/t_m)+((idx_max_test_hdf-idx_max_test_smc)/t_m)/2)
    g_coup_crestas = (((idx_max_test_ehz-idx_max_test_smc)/t_m)+((idx_max_test_hdf-idx_max_test_smc)/t_m)/2)
    print ("----------------------------------  grand coupling -----------: ",event__ini-seismic__ini)
    print ("----------------------------------  grand coupling crestas----: ",(((idx_max_test_ehz-idx_max_test_smc)/t_m)+((idx_max_test_hdf-idx_max_test_smc)/t_m)/2))
    
    if  ((event__ini-seismic__ini >=0 ) or (g_coup_crestas >0 ) )and ((((kurt_corr_ehz_smc_hdf_smc>1.5) and # or kurt_corr_ehz_smc_hdf_smc <0) and
          (kurt_hdf_smc>2) and              
          (kurt_ehz_smc>2) 
         ) or
        ((kurt_corr_ehz_smc_hdf_smc>1.1) and 
          (kurt_hdf_smc>1.5) and             
          (kurt_ehz_smc>5) 
         ) or
        ((kurt_corr_ehz_smc_hdf_smc>1.1) and 
          (kurt_hdf_smc>5) and              
          (kurt_ehz_smc>1.5) 
         ) or
        ((kurt_corr_ehz_smc_hdf_smc>1.4) and 
          (kurt_hdf_smc>5) and              
          (kurt_ehz_smc>1.1) 
         )or
        ((kurt_corr_ehz_smc_hdf_smc>1.4) and 
          (kurt_hdf_smc>5) and              
          (kurt_ehz_smc>1.1) 
         ) or
         (kurt_corr_ehz_smc_hdf_smc>1.6)
        )and              
         density_corr_ehz_smc_hdf_smc(-0.5) <0.25 and density_corr_ehz_smc_hdf_smc(0.5) <0.25): 
        
    # Imprimir los resultados de los cálculos
        conteo_explosiones_volcanicas +=1
        indice_explosiones_volcanicas.append(i)
        print("Kurtosis de la correlación cruzada (EHZ - SMC): ", kurt_ehz_smc)
        print("Kurtosis de la correlación cruzada (HDF - SMC): ", kurt_hdf_smc)
        print("Kurtosis de la correlación cruzada (EHZ-SMC - HDF-SMC): ", kurt_corr_ehz_smc_hdf_smc)
        print("Asimetría de la correlación cruzada (EHZ - SMC): ", skewness_ehz_smc)
        print("Asimetría de la correlación cruzada (HDF - SMC): ", skewness_hdf_smc)
        print("Asimetría de la correlación cruzada (EHZ-SMC - HDF-SMC): ", skewness_corr_ehz_smc_hdf_smc)
        print("Densidad de la correlacion cruzada (EHZ - SMC): ",density_ehz_smc(0.5))
        print("Densidad de la correlacion cruzada (HDF - SMC): ",density_hdf_smc(0.5))
        print("Densidad de la correlacion cruzada (EHZ-SMC - HDF-SMC): ",density_corr_ehz_smc_hdf_smc(0.5))

        print("Índice del máximo en EHZ: ", idx_max_test_ehz)
        print("Índice del máximo en SMC: ", idx_max_test_smc)
        print("Índice del máximo en HDF: ", idx_max_test_hdf)
        print("Grandcoupling EHZ-SMC -segundos-:", (idx_max_test_ehz-idx_max_test_smc)/t_m)
        print("Grandcoupling HDF-SMC -segundos-:", (idx_max_test_hdf-idx_max_test_smc)/t_m)
        print("Índice del máximo en la correlación cruzada (EHZ - SMC): ", idx_max_corr_ehz_smc)
        print("Índice del máximo en la correlación cruzada (HDF - SMC): ", idx_max_corr_hdf_smc)
        print("Índice del máximo en la correlación cruzada (EHZ-SMC - HDF-SMC): ", idx_max_corr_corr_ehz_smc_hdf_smc)

        # muestro el plot de las tres señales    
        test_ehz.plot()
        test_hdf.plot()
        test_smc.plot()

        plt.plot(np.linspace(min(corr_ehz_hdf), max(corr_ehz_hdf), 100), density_ehz_hdf(np.linspace(min(corr_ehz_hdf), max(corr_ehz_hdf), 100)), label='Densidad (IST - ISA)', color='green')
        plt.xlabel('Valor de correlación', fontsize=14)
        plt.ylabel('Densidad', fontsize=14)
        plt.title('Densidad de la correlación cruzada (IST - ISA)', fontsize=14)
        plt.legend(fontsize=14)

        # Ajustar el tamaño de la fuente de los ticks de los ejes
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.show()


    # Crear una figura para trazar las correlaciones cruzadas y las densidades
        fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(6, 1, figsize=(25, 45))

    # Trazar la correlación cruzada entre test_ehz.data y test_smc.data
        
        # Ajustar los márgenes entre los subplots
        plt.subplots_adjust(hspace=1)  # Puedes ajustar el valor 0.4 según tus necesidades
        ax1.plot(corr_ehz_smc, label='Correlación cruzada (IST - SMP)', color='green')
        ax1.set_xlabel('Desplazamiento', fontsize=30)
        ax1.set_ylabel('Valor de correlación', fontsize=30)
        ax1.set_title('Correlación cruzada normalizada entre IST y SMP', fontsize=25)
        ax1.legend(fontsize=25)
        ax1.tick_params(axis='both', which='major', labelsize=25)

    # Trazar la correlación cruzada entre test_hdf.data y test_smc.data
        ax2.plot(corr_hdf_smc, label='Correlación cruzada (ISA - SMP)', color='blue')
        ax2.set_xlabel('Desplazamiento', fontsize=30)
        ax2.set_ylabel('Valor de correlación', fontsize=30)
        ax2.set_title('Correlación cruzada normalizada entre ISA y SMP', fontsize=25)
        ax2.legend(fontsize=25)
        ax2.tick_params(axis='both', which='major', labelsize=25)

    # Trazar la correlación cruzada entre corr_ehz_smc y corr_hdf_smc
        ax3.plot(corr_corr_ehz_smc_hdf_smc, label='Correlación cruzada (IST-SMP - ISA-SMP)', color='purple')
        ax3.set_xlabel('Desplazamiento', fontsize=30)
        ax3.set_ylabel('Valor de correlación', fontsize=30)
        ax3.set_title('Correlación cruzada normalizada entre (IST-SMP) e (ISA-SMP)', fontsize=25)
        ax3.legend(fontsize=25)
        ax3.tick_params(axis='both', which='major', labelsize=25)

    # Trazar la densidad de la correlación cruzada (EHZ - SMC)
        x1 = np.linspace(min(corr_ehz_smc), max(corr_ehz_smc), 100)
        ax4.plot(x1, density_ehz_smc(x1), label='Densidad (IST - SMP)', color='green')
        ax4.set_xlabel('Valor de correlación', fontsize=30)
        ax4.set_ylabel('Densidad', fontsize=30)
        ax4.set_title('Densidad de la correlación cruzada (IST - SMP)', fontsize=25)
        ax4.legend(fontsize=25)
        ax4.tick_params(axis='both', which='major', labelsize=25)

    # Trazar la densidad de la correlación cruzada (HDF - SMC)
        x2 = np.linspace(min(corr_hdf_smc), max(corr_hdf_smc), 100)
        ax5.plot(x2, density_hdf_smc(x2), label='Densidad (ISA - SMP)', color='blue')
        ax5.set_xlabel('Valor de correlación', fontsize=30)
        ax5.set_ylabel('Densidad', fontsize=30)
        ax5.set_title('Densidad de la correlación cruzada (ISA - SMP)', fontsize=25)
        ax5.legend(fontsize=25)
        ax5.tick_params(axis='both', which='major', labelsize=25)

    # Trazar la densidad de la correlación cruzada (EHZ-SMC - HDF-SMC)
        x3 = np.linspace(min(corr_corr_ehz_smc_hdf_smc), max(corr_corr_ehz_smc_hdf_smc), 100)
        ax6.plot(x3, density_corr_ehz_smc_hdf_smc(x3), label='Densidad (IST-SMP - ISA-SMP)', color='purple')
        ax6.set_xlabel('Valor de correlación', fontsize=30)
        ax6.set_ylabel('Densidad', fontsize=30)
        ax6.set_title('Densidad de la correlación cruzada (IST-SMP - ISA-SMP)', fontsize=25)
        ax6.legend(fontsize=25)
        ax6.tick_params(axis='both', which='major', labelsize=25)

        # Ajustar el tamaño de la fuente de los ticks de los ejes
        plt.xticks(fontsize=25)
        plt.yticks(fontsize=25)
        plt.show()

    
        # Crear una figura
        fig, (ax_hdf, ax_ehz, ax_seismic) = plt.subplots(3, 1, sharex=True, figsize=(25, 45))
        ax_hdf.plot(tstmp_hdf,test_hdf.data, c='orange', alpha=0.5)
        ax_hdf.set_title('Onda acústica', fontsize=30)
        ax_ehz.plot(tstmp_ehz,test_ehz.data, c='blue', alpha=0.5)
        ax_ehz.set_title('Onda sismo-acústica', fontsize=30)
        ax_seismic.plot(tstmp_smc,test_smc.data, c='green', alpha=0.5)
        ax_seismic.set_title('Onda sísmica', fontsize=30)
        fig.tight_layout()
        
        # Ajustar el tamaño de la fuente de los ticks de los ejes
        plt.xticks(fontsize=25)
        plt.yticks(fontsize=25)
        plt.show()

#------------
        fig, ax1 = plt.subplots(figsize=(30, 15))

    # Trazar tstmp_ehz y test_ehz.data en el primer eje
        ax1.plot(tstmp_ehz, test_ehz.data, c='orange', alpha=1, label='Onda acústica detectada por sismómetro')
        ax1.set_xlabel('Tiempo (50 puntos por segundo)', fontsize=35)
        ax1.set_ylabel('Amplitud (cuentas digitales) - Acústica-Sismica', fontsize=35, color='black')
        ax1.tick_params(axis='y', labelcolor='black')
        ax1.legend(fontsize=35, loc='upper left')
        ax1.tick_params(axis='y', labelcolor='black', labelsize=35)
        ax1.tick_params(axis='x', labelsize=35)
        ax1.legend(fontsize=35, loc='upper left')

    # Crear un segundo eje compartiendo el mismo eje x
        ax2 = ax1.twinx()

    # Trazar tstmp_hdf y test_hdf.data en el segundo eje
        ax2.plot(tstmp_hdf, test_hdf.data, c='blue', alpha=0.7, label='Onda acústica detectada por microbarómetro')
        ax2.set_ylabel('Amplitud (cuentas digitales) - Acústica-Atmosférica', fontsize=35, color='blue')
        ax2.tick_params(axis='y', labelcolor='black')
        ax2.legend(fontsize=35, loc='upper right')
        ax2.tick_params(axis='y', labelcolor='blue', labelsize=35)
        ax2.legend(fontsize=35, loc='upper right')
        
    # Crear un tercer eje compartiendo el mismo eje x
        ax3 = ax1.twinx()
        
    # Trazar tstmp_smc y test_smc.data en el tercer eje
        ax3.plot(tstmp_smc, test_smc.data, c='green', alpha=0.5, label='Onda sísmica')
        ax3.set_ylabel('Amplitud (cuentas digitales) - Sísmica', fontsize=35, color='black')
        ax3.tick_params(axis='y', labelcolor='black')
        ax3.legend(fontsize=35, loc='lower right')

        # Desplazar el tercer eje a la derecha
        ax3.spines['right'].set_position(('outward', 160))  # Ajusta el valor según necesites
        
    # Agregar líneas horizontales en cero a cada eje
        ax1.axhline(0, color='black', linestyle='--', linewidth=1)
        ax2.axhline(0, color='black', linestyle='--', linewidth=1)
        ax3.axhline(0, color='black', linestyle='--', linewidth=1)
        
        
        # Ajustar el tamaño de la fuente de los ticks de los ejes
        plt.xticks(fontsize=25)
        plt.yticks(fontsize=25)
        plt.show()        
        
    else:
        print("Kurtosis de la correlación cruzada (EHZ - SMC): ", kurt_ehz_smc)
        print("Kurtosis de la correlación cruzada (HDF - SMC): ", kurt_hdf_smc)
        print("Kurtosis de la correlación cruzada (EHZ-SMC - HDF-SMC): ", kurt_corr_ehz_smc_hdf_smc)
        print("Densidad de la correlacion cruzada (EHZ-SMC - HDF-SMC): (-0.5) ",density_corr_ehz_smc_hdf_smc(-0.5))
        print("Densidad de la correlacion cruzada (EHZ-SMC - HDF-SMC): (.5) ",density_corr_ehz_smc_hdf_smc(0.5))
        indice_explosiones_volcanicas.append("")
print("")

print ("Cantidad de explosiones volcánicas detectadas:", conteo_explosiones_volcanicas)



In [ ]:
df_intervalos_2['explosion_volcanica'] = indice_explosiones_volcanicas
df_intervalos_2['grand_coupling_crestas'] = grand_coupling_actualizado
df_intervalos_2.to_excel("df_intervalos_2.xlsx")

# Tomo recortes del catálogo base del Insivumeh

In [ ]:
import pandas as pd

archivo_excel = "Catálogo_Explosiones_Fuego_17092022.xlsx"

# Leer el archivo Excel
data = pd.read_excel(archivo_excel, header=None, names=['fecha', 'No. de serie', 'estado'])

# Descartar la segunda columna
data.drop(columns=['No. de serie'], inplace=True)

# Crear una lista vacía para almacenar las marcas de tiempo
marcas_de_tiempo = []

var_Ex  = pd.NaT
var_ExG = pd.NaT
var_NS  = pd.NaT

# Iterar sobre los datos originales
for index, row in data.iterrows():
    if row.estado == "Ex":
        var_Ex = UTC(row.fecha)
    if row.estado == "ExG":
        var_ExG = UTC(row.fecha)
    if row.estado == "NS":
        var_NS = UTC(row.fecha)
        # Agregar registros vacíos a la lista de marcas de tiempo
        marcas_de_tiempo.append({'Ex': var_Ex, 'ExG': var_ExG, 'NS': var_NS})
        var_Ex  = pd.NaT
        var_ExG = pd.NaT
        var_NS  = pd.NaT

# Imprimir la lista resultante
for evento in marcas_de_tiempo:
    print(evento)
    
# Convertir la lista de marcas de tiempo en un DataFrame
df_EV_BASE = pd.DataFrame(marcas_de_tiempo)

print(df_EV_BASE)


# Guardo archivos mseed de los recortes

In [ ]:
st__HDF = read("GI.FG12.02.BDF.D.2022.260.mseed")
st__EHZ = read("GI.FG12.00.BHZ.D.2022.260.mseed")
st__SMC = read("GI.FG12.00.BHZ.D.2022.260.mseed")

fmin_HDF=0.5
fmax_HDF=20
fmin_EHZ = 5
fmax_EHZ = 15
fmin_seismic = 0.2
fmax_seismic = 7

st_HDF   = st__HDF.filter('bandpass', freqmin= fmin_HDF, freqmax=fmax_HDF, corners=2, zerophase=True)
st_EHZ = st__EHZ.filter('bandpass', freqmin= fmin_EHZ, freqmax=fmax_EHZ, corners=2, zerophase=True)
st_smc = st__SMC.filter('highpass', freq= fmin_seismic,                       corners=8, zerophase=True)
st_smc = st_smc
tr_HDF = st_HDF[0]
tr_EHZ = st_EHZ[0]

# Define una lista de colores para las trazas
colors = ["red", "blue"]
for i, eventos_row in df_EV_BASE.iterrows():
    print(eventos_row)
    AAW_test = st_HDF.slice(eventos_row.Ex, eventos_row.NS)
    GAW_test = st_EHZ.slice(eventos_row.Ex, eventos_row.NS)
    seismic_test = st_smc.slice(eventos_row.Ex, eventos_row.NS)

    plt.figure(figsize=(10, 8))
    AAW_test.plot(color = "red")
    GAW_test.plot(color = "blue")
    seismic_test.plot(color="green")
    
    template_dir = "slices"
    if not os.path.exists(template_dir):
        os.makedirs(template_dir)
    AAW_test.write(os.path.join(template_dir, f"{eventos_row.name}_AAW.mseed"), format="MSEED")
    GAW_test.write(os.path.join(template_dir, f"{eventos_row.name}_GAW.name.mseed"), format="MSEED")
    seismic_test.write(os.path.join(template_dir, f"{eventos_row.name}_seismic.mseed"), format="MSEED")

# Se muestran los espectros de las explociones del catalogo del Insivumeh

In [ ]:
import os
from obspy import read

# Directorio donde se encuentran los archivos MSEED
directory = "slices"

# Verifica si el directorio existe
if not os.path.exists(directory):
    print(f"El directorio '{directory}' no existe.")
    exit()

# Obtén la lista de archivos MSEED en el directorio
mseed_files = [file for file in os.listdir(directory) if file.endswith(".mseed")]
print(mseed_files)
# Itera sobre cada archivo MSEED
for file in mseed_files:
    # Lee el archivo MSEED
    st = read(os.path.join(directory, file))
    
    # Verifica si el archivo es sísmico o acústico
    if "seismic" in file:
        # Realiza las operaciones necesarias con el evento sísmico
        print(f"Evento sísmico: {file}")
        for trace in st:
            print(trace)
            trace.plot()
            
            # Realiza la transformada de Fourier
            spectrum_seismic = np.fft.fft(trace.data)
        
            # Calcula las frecuencias correspondientes
            freq_seismic = np.fft.fftfreq(len(trace.data), d=1/trace.stats.sampling_rate)
        
            # Grafica el espectro
            plt.figure()
            plt.plot(np.abs(freq_seismic), np.abs(spectrum_seismic))
            plt.xlabel('Frecuencia (Hz)', fontsize=15)
            plt.ylabel('Amplitud', fontsize=15)
            plt.title('Espectro de onda sismica ' + trace.stats.station + ' - ' + trace.stats.channel,fontsize=12)
            plt.grid(True)
            
            # Ajustar el tamaño de la fuente de los ticks de los ejes
            plt.xticks(fontsize=15)
            plt.yticks(fontsize=15)
            plt.show()

    
    if "AAW" in file:
        # Realiza las operaciones necesarias con el evento sísmico
        print(f"Evento acústico atmosférico: {file}")
        for trace in st:
            print(trace)
            trace.plot()

            # Realiza la transformada de Fourier
            spectrum_AAW = np.fft.fft(trace.data)
        
            # Calcula las frecuencias correspondientes
            freq_AAW = np.fft.fftfreq(len(trace.data), d=1/trace.stats.sampling_rate)
        
            # Grafica el espectro
            plt.figure()
            plt.plot(np.abs(freq_AAW), np.abs(spectrum_AAW))
            plt.xlabel('Frecuencia (Hz)')
            plt.ylabel('Amplitud')
            plt.title('Espectro de onda acústica atmosférica ' + trace.stats.station + ' - ' + trace.stats.channel,fontsize=12)
            plt.grid(True)
            # Ajustar el tamaño de la fuente de los ticks de los ejes
            plt.xticks(fontsize=15)
            plt.yticks(fontsize=15)
            plt.show()
                        
    else:
        # Realiza las operaciones necesarias con los eventos acústicos
        print(f"Evento acústico terrestre: {file}")
        for trace in st:
            print(trace)
            trace.plot()
            # Realiza la transformada de Fourier
            spectrum_GAW = np.fft.fft(trace.data)
        
            # Calcula las frecuencias correspondientes
            freq_GAW = np.fft.fftfreq(len(trace.data), d=1/trace.stats.sampling_rate)
        
            # Grafica el espectro
            plt.figure()
            plt.plot(np.abs(freq_GAW), np.abs(spectrum_GAW))
            plt.xlabel('Frecuencia (Hz)')
            plt.ylabel('Amplitud')
            plt.title('Espectro de onda acústica captada por el sismómetro ' + trace.stats.station + ' - ' + trace.stats.channel,fontsize=12)
            plt.grid(True)
            # Ajustar el tamaño de la fuente de los ticks de los ejes
            plt.xticks(fontsize=15)
            plt.yticks(fontsize=15)
            plt.show()
    print("\n")




# Se crea un arreglo con los espectros de las explociones del catalogo del Insivumeh

In [ ]:
import os
from obspy import read
import numpy as np
import matplotlib.pyplot as plt

# Directorio donde se encuentran los archivos MSEED
directory = "slices"

# Verifica si el directorio existe
if not os.path.exists(directory):
    print(f"El directorio '{directory}' no existe.")
    exit()

# Obtén la lista de archivos MSEED en el directorio
mseed_files = [file for file in os.listdir(directory) if file.endswith(".mseed")]

# Listas para almacenar los espectros y frecuencias
spectra_seismic = []
freqs_seismic = []
signal_seismic = []
spectra_AAW = []
freqs_AAW = []
signal_AAW = []

spectra_GAW = []
freqs_GAW = []
signal_GAW = []

# Itera sobre cada archivo MSEED
for file in mseed_files:
    # Lee el archivo MSEED
    st = read(os.path.join(directory, file))
    
    # Itera sobre cada traza en el archivo MSEED
    for trace in st:
        # Realiza la transformada de Fourier
        spectrum = np.fft.fft(trace.data)
        
        # Calcula las frecuencias correspondientes
        freq = np.fft.fftfreq(len(trace.data), d=1/trace.stats.sampling_rate)
        
        # Verifica el tipo de onda y agrega el espectro y las frecuencias correspondientes a las listas adecuadas
        if "seismic" in file:
            signal_seismic.append(trace.data)
            spectra_seismic.append(np.abs(spectrum))
            freqs_seismic.append(freq)
        elif "AAW" in file:
            signal_AAW.append(trace.data)
            spectra_AAW.append(np.abs(spectrum))
            freqs_AAW.append(freq)
        else:
            signal_GAW.append(trace.data)
            spectra_GAW.append(np.abs(spectrum))
            freqs_GAW.append(freq)
# Ahora se tienen los espectros y frecuencias de cada tipo de onda en listas separadas
# Ahora se puede comparar estas listas como desees


#  Función que realiza las correlaciones cruzadas del dominio de la frecuencia entre un candidato a explosión volcánica y las explosiones confirmadas del catálogo

In [ ]:
def calcular_correlaciones_cruzadas_dominio_frecuencia(signal_test, spectrum_test, freq_test, signal_list, spectrum_list, freq_list):
    kurt_freq = []
    kurt_spectrum = []
    kurt_signal = []

    corr_sg = []
    corr_f = []
    corr_s = []

    dens_sg = []
    dens_f = []
    dens_s = []
    
    for signal_d,spectrum, freq in zip(signal_list,spectrum_list, freq_list):
        norm_spectrum_test = np.abs(spectrum_test) / np.max(np.abs(spectrum_test))
        norm_spectrum = np.abs(spectrum) / np.max(np.abs(spectrum))
        
        norm_freq_test = freq_test / np.max(freq_test)
        norm_freq = freq / np.max(freq)
        
        # Calcula la correlación cruzada entre los espectros y las frecuencias normalizadas
        correlation_signal = signal.correlate(signal_test, signal_d, mode='same', method='fft')
        correlation_spectrum = signal.correlate(norm_spectrum_test, norm_spectrum, mode='same', method='fft')#np.correlate(norm_spectrum_test, norm_spectrum)
        correlation_freq = signal.correlate(norm_freq_test, norm_freq, mode='same', method='fft')#np.correlate(norm_freq_test, norm_freq)

        correlation_signal /= np.max(np.abs(correlation_signal))
        correlation_spectrum /= np.max(np.abs(correlation_spectrum))
        correlation_freq /= np.max(np.abs(correlation_freq))
        
        corr_sg.append( correlation_signal)
        corr_f.append( correlation_freq)
        corr_s.append( correlation_spectrum)
        
        signal_densidad = gaussian_kde(correlation_signal)
        freq_densidad     =gaussian_kde(correlation_freq)
        spectrum_densidad =gaussian_kde(correlation_spectrum)
        
        dens_sg.append(signal_densidad)
        dens_f.append(freq_densidad)
        dens_s.append(spectrum_densidad)
        
        k_sg = kurtosis(correlation_signal)
        k_f =  kurtosis(correlation_freq)
        k_s = kurtosis(correlation_spectrum)

        kurt_signal.append(k_sg)
        kurt_freq.append   (k_f)
        kurt_spectrum.append (k_s)
        
    print(Fore.BLUE + "        ===============Correlación cruzada normalizada entre spectrum test y list===============" + Style.RESET_ALL)
    plt.plot(corr_s[np.argmax(kurt_spectrum)], label='Correlación cruzada spectrum (test-list)', color='orange')
    plt.xlabel('Desplazamiento', fontsize=15)
    plt.ylabel('Valor de correlación', fontsize=15)

    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()

    print(Fore.BLUE + "        ===============Correlación cruzada normalizada entre signal test y list===============" + Style.RESET_ALL)
    plt.plot(corr_sg[np.argmax(kurt_signal)], label='Correlación cruzada signal (test-list)', color='orange')
    plt.xlabel('Desplazamiento', fontsize=15)
    plt.ylabel('Valor de correlación', fontsize=15)

    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()

    print(Fore.BLUE + "        ===============Densidad de la correlación cruzada spectrum test y list===============" + Style.RESET_ALL)
    x1 = np.linspace(min(correlation_spectrum), max(correlation_spectrum), 100)
    plt.plot(x1, dens_s[np.argmax(kurt_spectrum)](x1), label='Densidad spectrum (test-list)', color='green')
    plt.xlabel('Valor de correlación', fontsize=15)
    plt.ylabel('Densidad', fontsize=15)

    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()

    print(Fore.BLUE + "        ===============Densidad de la correlación cruzada signal test y list===============" + Style.RESET_ALL)
    x1 = np.linspace(min(correlation_signal), max(correlation_signal), 100)
    plt.plot(x1, dens_sg[np.argmax(kurt_signal)](x1), label='Densidad signal (test-list)', color='green')
    plt.xlabel('Valor de correlación', fontsize=15)
    plt.ylabel('Densidad', fontsize=15)

    # Ajustar el tamaño de la fuente de los ticks de los ejes
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.show()

    # Imprime los resultados
    print("kurtosis de la correlación cruzada de señales:", kurt_signal[np.argmax(kurt_signal)])
    print("kurtosis de la correlación cruzada de espectros:", kurt_spectrum[np.argmax(kurt_spectrum)])
    print("kurtosis de la correlación cruzada de frecuencias:", kurt_freq[np.argmax(kurt_freq)])
    print("Densidad de la correlacion cruzada spectrum (0): ",dens_s[np.argmax(kurt_spectrum)](0))
    print("Densidad de la correlacion cruzada spectrum (1): ",dens_s[np.argmax(kurt_spectrum)](1))
    print("Densidad de la correlacion cruzada señal (-0.5): ",dens_sg[np.argmax(kurt_signal)](-0.5))
    print("Densidad de la correlacion cruzada señal (0.5): ",dens_sg[np.argmax(kurt_signal)](0.5))
        
    print("-" * 40)
    return kurt_spectrum[np.argmax(kurt_spectrum)], kurt_signal[np.argmax(kurt_signal)]

# Se realizan las correlaciones con los recortes de prueba del catalogo del insivumeh

In [ ]:
from colorama import Fore, Style, init

# Inicializar colorama
init()
st__HDF = read("GI.FG12.02.BDF.D.2022.260.mseed")
st__EHZ = read("GI.FG12.00.BHZ.D.2022.260.mseed")
st__SMC = read("GI.FG12.00.BHZ.D.2022.260.mseed")

fmin_HDF=0.5
fmax_HDF=20
fmin_EHZ = 5#0.5 #5 #.5
fmax_EHZ = 15#5   #15 #7
fmin_seismic = 0.2
fmax_seismic = 7

st_HDF   = st__HDF.filter('bandpass', freqmin= fmin_HDF, freqmax=fmax_HDF, corners=2, zerophase=True)
st_EHZ = st__EHZ.filter('bandpass', freqmin= fmin_EHZ, freqmax=fmax_EHZ, corners=2, zerophase=True)
st_smc = st__SMC.filter('highpass', freq= fmin_seismic,                       corners=8, zerophase=True)
tr_HDF = st_HDF[0]
tr_EHZ = st_EHZ[0]

# Define una lista de colores para las trazas
colors = ["red", "blue"]

caso_1 =0
caso_2 =0
caso_3 =0
caso_4 =0
confirmed = 0
indice_confirmacion = []

df_intervalos_2 = pd.read_excel("df_intervalos_2.xlsx")
for i, eventos_row in df_intervalos_2.iterrows():
    event__ini = UTC(eventos_row.event_ini)
    event__fin = UTC(eventos_row.event_fin)
    seismic__ini = UTC(eventos_row.seismic_ini)
    seismic__fin = UTC(eventos_row.seismic_fin)

    if  eventos_row.explosion_volcanica >0 and seismic__ini <= event__ini and  seismic__fin >= event__fin and eventos_row.explosion_volcanica >0:
        print("")
        print("---------------------------------------------------- ----------------------------------------------------")
        print("aplico s[ e[ e] s]")
        print (eventos_row.explosion_volcanica, "seismic_ini", seismic__ini, "event_ini", event__ini,"event_fin", event__fin, "seismic fin", seismic__fin)

        AAW_test = st_HDF.slice(seismic__ini, seismic__fin)
        GAW_test = st_EHZ.slice(seismic__ini, seismic__fin)
        seismic_test = st_smc.slice(seismic__ini, seismic__fin)
        
        AAW_test.plot(color = "red")
        GAW_test.plot(color = "blue")
        seismic_test.plot(color="green")

         # Realiza la transformada de Fourier
        spectrum_seismic_test = np.fft.fft(seismic_test[0].data)
        spectrum_AAW_test = np.fft.fft(AAW_test[0].data)
        spectrum_GAW_test = np.fft.fft(GAW_test[0].data)

        # Calcula las frecuencias correspondientes
        freq_seismic_test = np.fft.fftfreq(len(seismic_test[0].data), d=1/seismic_test[0].stats.sampling_rate)
        freq_AAW_test = np.fft.fftfreq(len(AAW_test[0].data), d=1/AAW_test[0].stats.sampling_rate)
        freq_GAW_test = np.fft.fftfreq(len(GAW_test[0].data), d=1/GAW_test[0].stats.sampling_rate)
        
        caso_1+=1
    
    elif eventos_row.explosion_volcanica >0 and seismic__ini <= event__ini and  seismic__fin <= event__fin and seismic__fin> event__ini  and eventos_row.explosion_volcanica >0:
        print("")
        print("---------------------------------------------------- ----------------------------------------------------")
        print("aplico s[ e[ s] e]")
        print (eventos_row.explosion_volcanica, "seismic_ini", seismic__ini, "event_ini", event__ini,"seismic_fin", event__fin, "event fin", seismic__fin)

        AAW_test = st_HDF.slice(seismic__ini, event__fin)
        GAW_test = st_EHZ.slice(seismic__ini, event__fin)
        seismic_test = st_smc.slice(seismic__ini, event__fin)
        
        AAW_test.plot(color = "red")
        GAW_test.plot(color = "blue")
        seismic_test.plot(color="green")

        # Realiza la transformada de Fourier
        spectrum_seismic_test = np.fft.fft(seismic_test[0].data)
        spectrum_AAW_test = np.fft.fft(AAW_test[0].data)
        spectrum_GAW_test = np.fft.fft(GAW_test[0].data)

        # Calcula las frecuencias correspondientes
        freq_seismic_test = np.fft.fftfreq(len(seismic_test[0].data), d=1/seismic_test[0].stats.sampling_rate)
        freq_AAW_test = np.fft.fftfreq(len(AAW_test[0].data), d=1/AAW_test[0].stats.sampling_rate)
        freq_GAW_test = np.fft.fftfreq(len(GAW_test[0].data), d=1/GAW_test[0].stats.sampling_rate)
        
        caso_2+=1

    elif eventos_row.explosion_volcanica >0 and seismic__ini >= event__ini and  seismic__fin >= event__fin and event__fin> seismic__ini and eventos_row.explosion_volcanica >0:
        print("")
        print("---------------------------------------------------- ----------------------------------------------------")
        print("aplico e[ s[ e] s]")
        print (eventos_row.explosion_volcanica, "event_ini", event__ini, "event_ini", seismic__ini,"event_fin", event__fin, "seismic fin", seismic__fin)
        
        AAW_test = st_HDF.slice(event__ini, seismic__fin)
        GAW_test = st_EHZ.slice(event__ini, seismic__fin)
        seismic_test = st_smc.slice(event__ini, seismic__fin)
        
        AAW_test.plot(color = "red")
        GAW_test.plot(color = "blue")
        seismic_test.plot(color="green")
 
        # Realiza la transformada de Fourier
        spectrum_seismic_test = np.fft.fft(seismic_test[0].data)
        spectrum_AAW_test = np.fft.fft(AAW_test[0].data)
        spectrum_GAW_test = np.fft.fft(GAW_test[0].data)

        # Calcula las frecuencias correspondientes
        freq_seismic_test = np.fft.fftfreq(len(seismic_test[0].data), d=1/seismic_test[0].stats.sampling_rate)
        freq_AAW_test = np.fft.fftfreq(len(AAW_test[0].data), d=1/AAW_test[0].stats.sampling_rate)
        freq_GAW_test = np.fft.fftfreq(len(GAW_test[0].data), d=1/GAW_test[0].stats.sampling_rate)
        
        caso_3 +=1
 
    elif  eventos_row.explosion_volcanica >0:
        print("")
        print("---------------------------------------------------- ----------------------------------------------------")
        print("evento", i, "aplico { e[ e] s[ s] } o { s[ s] e[e] }")
        print (eventos_row.explosion_volcanica, "seismic_ini", seismic__ini, "seismic fin", seismic__fin, "event_ini", event__ini,"event_fin", event__fin)
        min_ini = min(event__ini, seismic__ini)
        max_fin = max(event__fin, seismic__fin)

        AAW_test = st_HDF.slice(min_ini, max_fin)
        GAW_test = st_EHZ.slice(min_ini, max_fin)
        seismic_test = st_smc.slice(min_ini, max_fin)
        
        AAW_test.plot(color = "red")
        GAW_test.plot(color = "blue")
        seismic_test.plot(color="green")
        
        # Realiza la transformada de Fourier
        spectrum_seismic_test = np.fft.fft(seismic_test[0].data)
        spectrum_AAW_test = np.fft.fft(AAW_test[0].data)
        spectrum_GAW_test = np.fft.fft(GAW_test[0].data)

        # Calcula las frecuencias correspondientes
        freq_seismic_test = np.fft.fftfreq(len(seismic_test[0].data), d=1/seismic_test[0].stats.sampling_rate)
        freq_AAW_test = np.fft.fftfreq(len(AAW_test[0].data), d=1/AAW_test[0].stats.sampling_rate)
        freq_GAW_test = np.fft.fftfreq(len(GAW_test[0].data), d=1/GAW_test[0].stats.sampling_rate)
        
        caso_4 +=1


    # Llama a la función para cada tipo de onda
    if  eventos_row.explosion_volcanica >0: 
        print("Comparación con seismic:")
        k_spc_seismic, k_sgn_seismic = calcular_correlaciones_cruzadas_dominio_frecuencia(seismic_test[0].data,spectrum_seismic_test, freq_seismic_test, signal_seismic, spectra_seismic, freqs_seismic)

        print("Comparación con AAW:")
        k_spc_AAW, k_sgn_AAW = calcular_correlaciones_cruzadas_dominio_frecuencia(AAW_test[0].data,spectrum_AAW_test, freq_AAW_test, signal_AAW,spectra_AAW, freqs_AAW)

        print("Comparación con GAW:")
        k_spc_GAW, k_sgn_GAW = calcular_correlaciones_cruzadas_dominio_frecuencia(GAW_test[0].data,spectrum_GAW_test, freq_GAW_test, signal_GAW,spectra_GAW, freqs_GAW)
        print('kurt espectro',k_spc_seismic, k_spc_AAW, k_spc_GAW)
        print('kurt_señal', k_sgn_seismic, k_sgn_AAW, k_sgn_GAW)
        if (k_spc_seismic >= 1.6 or k_spc_AAW >= 1.5 or k_spc_GAW > 1.5) and (k_sgn_seismic >= 5 or k_sgn_AAW >= 5 or k_sgn_GAW > 5):
            print(Fore.RED + "===============EXPLOSION VOLCÁNICA CONFIMADA===============" + Style.RESET_ALL)
            
            confirmed +=1
            indice_confirmacion.append(eventos_row.explosion_volcanica)
        # Grafica el espectro
            print(Fore.BLUE + "        ===============Espectro de onda sismica===============" + Style.RESET_ALL)
            plt.figure()
            plt.plot(np.abs(freq_seismic_test), np.abs(spectrum_seismic_test))
            plt.xlabel('Frecuencia (Hz)', fontsize=15)
            plt.ylabel('Amplitud', fontsize=15)
#            plt.title('Espectro de onda sismica ', fontsize=25 )
            plt.grid(True)
            # Ajustar el tamaño de la fuente de los ticks de los ejes
            plt.xticks(fontsize=14)
            plt.yticks(fontsize=14)
            plt.show()
        # Grafica el espectro
            print(Fore.BLUE + "        ===============Espectro de onda acustica atmosférica===============" + Style.RESET_ALL)
            plt.figure()
            plt.plot(np.abs(freq_AAW_test), np.abs(spectrum_AAW_test))
            plt.xlabel('Frecuencia (Hz)', fontsize=15)
            plt.ylabel('Amplitud', fontsize=15)
            plt.grid(True)

            # Ajustar el tamaño de la fuente de los ticks de los ejes
            plt.xticks(fontsize=14)
            plt.yticks(fontsize=14)
            plt.show()

        # Grafica el espectro
            print(Fore.BLUE + "        ===============Espectro de onda acústica captada por el sismómetro===============" + Style.RESET_ALL)
            plt.figure()
            plt.plot(np.abs(freq_GAW_test), np.abs(spectrum_GAW_test))
            plt.xlabel('Frecuencia (Hz)', fontsize=15)
            plt.ylabel('Amplitud', fontsize=15)
            plt.grid(True)

            # Ajustar el tamaño de la fuente de los ticks de los ejes
            plt.xticks(fontsize=14)
            plt.yticks(fontsize=14)
            plt.show()
        else:
            indice_confirmacion.append("")
    else:
        indice_confirmacion.append("")

print(caso_1,caso_2,caso_3,caso_4, confirmed)
df_intervalos_2['explosion_confirmada'] = indice_confirmacion
df_intervalos_2.to_excel("df_intervalos_2_confirmed.xlsx")